Imports

In [333]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, date, timedelta
from faker import Faker
from faker.providers import BaseProvider
from math import ceil

AMR Spreadsheet

AMRID -> process

SupplierID -> extract  + process

Assignedby -> person + process

DateAssigned -> process

Destination -> process

DateSubmitted -> extract

AmountAvailable -> extract

Synthesis -> should be n/a for extracts

SourceId -> extract

SourceType -> extract

Species -> Get from genomics or collection

Permit -> collection

Pure -> Yes for pure compounds, no for extracts

MS -> just link if there to process

NMR -> just link if there to process

ResearchGroup -> extract

Submittedby -> person + extract

Solubility -> n/a for extracts

Stereo comments -> n/a for extracts

Solvent for extraction -> extract

SMILE -> n/a for extract

MW -> n/a for extract

AdditionalMetadata -> removed

ExistingPatent -> n/a for extract

ExistingLiterature -> extract

Comments -> removed

In [334]:
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Robert Hobson', 'Julia Reed']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['Rhodes Bioassay', 'Rhodes MS', 'Rhodes Bioassay/Rhodes MS', 'Rhodes Bioassay, PK assay', 'Rhodes MS/NMR', 'Membrane Disruption assay', 'Rhodes LC/MS + NMR & Bioassay']
    start_date = datetime(2050, 1, 1)
    end_date = datetime(2055, 1, 1)
    binaryChoice = ['Y', 'N']
    existingLit = ['Y', 'N', 'https//doi.org/choice1', 'https//doi.org/choice2', 'https//doi.org/choice3', 'https//doi.org/choice4', 'https//doi.org/choice5', 'https//doi.org/choice6']
    researchgroups = ['Smith', 'Roodt', 'Wellington', 'Preston']
    solvents = ['DMSO', 'Methanol']

    def year(self):
        return random.randint(2041, 2049)

    def submittedDate(self):
        self.submittedDate = fake.date_between_dates(
            date_start=self.start_date, date_end=self.end_date).strftime('%d/%m/%Y')

        return self.submittedDate
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))

    def synthesisPotential(self):
        return "N/A"

    def species(self):
        return fake.word()

    def pure(self):
        return "N"

    def ms(self):
        return random.choice(self.binaryChoice)
    
    def nmr(self):
        return random.choice(self.binaryChoice)

    def researchGroup(self):
        return random.choice(self.researchgroups)

    def solubility(self):
        return "N/A"

    def stereoComments(self):
        return "N/A"

    def smileStructure(self):
        return "N/A"

    def mw(self):
        return "N/A"

    def additionalMetadata(self):
        return random.choice(self.binaryChoice)

    def existingPatent(self):
        return "N"

    def existingLiterature(self):
        return random.choice(self.existingLit)
    
    def solvent(self):
        return random.choice(self.solvents)

# Add the Provider to our faker object
fake.add_provider(Provider)

# Go up by one for AMR ID
# Supplier ID - based on date of collection, so must be same or earlier year than submittedDate
#               some TIC, some SAF, Some strains (e.g. S 20 20), MIAP_155, HMF-1, MBA_F406_PDM
# Date assigned - do a week after submittedDate
# Source ID - TIC number, MIAP etc - based on Supplier ID (start with this, then make Supplier ID)
# Source type - based on source ID (microbial vs sponge vs stromatolite)
# Permit - RES and year before or same as source ID

def create_AMR_data(fake, no_of_rows):
    columns = ['year', 'assignedBy', 'destinationOfSample', 'submittedDate', 'amountAvailable', 'synthesisPotential', 'species', 'pure', 'ms', 'nmr',
                'researchGroup', 'submittedBy', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature',
                'solvent']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

original_df = create_AMR_data(fake, 100)
original_df

,year,assignedBy,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,ms,nmr,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,2048,Ruby Johnson,Rhodes MS,01/10/2053,16.3,N/A,wrong,N,N,Y,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,2042,Joshua Keener,Rhodes MS,30/12/2050,49.9,N/A,next,N,Y,N,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,2041,Ruby Johnson,Rhodes MS/NMR,24/01/2050,94.0,N/A,board,N,N,Y,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,2042,Ruby Johnson,Rhodes MS/NMR,09/10/2050,95.0,N/A,themselves,N,Y,N,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,2041,Joshua Keener,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,though,N,Y,N,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2049,Joshua Keener,Rhodes MS/NMR,20/01/2051,75.5,N/A,chair,N,Y,N,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,2042,Ruby Johnson,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,catch,N,Y,Y,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,2048,Joshua Keener,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,anything,N,N,N,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,2047,Joshua Keener,Rhodes MS/NMR,16/09/2053,51.4,N/A,probably,N,N,N,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


Adding in AMR ID

In [335]:
# Go up by one for AMR ID
original_df.insert(0, 'amrID', range(1, 1 + len(original_df)))
original_df
print(original_df.dtypes)

amrID                    int64
year                     int64
assignedBy              object
destinationOfSample     object
submittedDate           object
amountAvailable        float64
synthesisPotential      object
species                 object
pure                    object
ms                      object
nmr                     object
researchGroup           object
submittedBy             object
solubility              object
stereoComments          object
smileStructure          object
mw                      object
additionalMetadata      object
existingPatent          object
existingLiterature      object
solvent                 object
dtype: object


In [336]:
original_df['amrID'] = 'AMR' + original_df['amrID'].astype(str).str.zfill(5)
print(original_df.dtypes)

amrID                   object
year                     int64
assignedBy              object
destinationOfSample     object
submittedDate           object
amountAvailable        float64
synthesisPotential      object
species                 object
pure                    object
ms                      object
nmr                     object
researchGroup           object
submittedBy             object
solubility              object
stereoComments          object
smileStructure          object
mw                      object
additionalMetadata      object
existingPatent          object
existingLiterature      object
solvent                 object
dtype: object


In [337]:
original_df

,amrID,year,assignedBy,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,ms,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,Rhodes MS,01/10/2053,16.3,N/A,wrong,N,N,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,Rhodes MS,30/12/2050,49.9,N/A,next,N,Y,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,Rhodes MS/NMR,24/01/2050,94.0,N/A,board,N,N,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,Rhodes MS/NMR,09/10/2050,95.0,N/A,themselves,N,Y,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,though,N,Y,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,Rhodes MS/NMR,20/01/2051,75.5,N/A,chair,N,Y,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,catch,N,Y,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,anything,N,N,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,Rhodes MS/NMR,16/09/2053,51.4,N/A,probably,N,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


Date Assigned column addition

In [338]:
# Date assigned - do a week after submittedDate
original_df.submittedDate = pd.to_datetime(original_df.submittedDate, format='%d/%m/%Y')
original_df.insert(3, 'assignedDate', original_df['submittedDate'])
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,2053-10-01,Rhodes MS,2053-10-01,16.3,N/A,wrong,N,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,2050-12-30,Rhodes MS,2050-12-30,49.9,N/A,next,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,2050-01-24,Rhodes MS/NMR,2050-01-24,94.0,N/A,board,N,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,2050-10-09,Rhodes MS/NMR,2050-10-09,95.0,N/A,themselves,N,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,2054-12-14,"Rhodes Bioassay, PK assay",2054-12-14,14.0,N/A,though,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,2051-01-20,Rhodes MS/NMR,2051-01-20,75.5,N/A,chair,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,2051-07-31,Rhodes Bioassay/Rhodes MS,2051-07-31,58.7,N/A,catch,N,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,2053-07-17,Rhodes LC/MS + NMR & Bioassay,2053-07-17,82.8,N/A,anything,N,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,2053-09-16,Rhodes MS/NMR,2053-09-16,51.4,N/A,probably,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


In [339]:

original_df['assignedDate'] = original_df['assignedDate'] + timedelta(days=7)
original_df['assignedDate'] = original_df['assignedDate'].dt.strftime('%d/%m/%Y')
original_df['submittedDate'] = original_df['submittedDate'].dt.strftime('%d/%m/%Y')
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,wrong,N,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,next,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,board,N,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,themselves,N,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,though,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,chair,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,catch,N,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,anything,N,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,probably,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


Adding Permit column

In [340]:
# Permit - RES and year before or same as source ID
original_df.insert(9, 'permitNo', original_df['year'].astype(str))
original_df['permitNo'] = 'RES' + original_df['permitNo']
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,permitNo,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,wrong,RES2048,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,next,RES2042,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,board,RES2041,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,themselves,RES2042,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,though,RES2041,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,chair,RES2049,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,catch,RES2042,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,anything,RES2048,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,probably,RES2047,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


Adding Source ID

In [341]:
# Source ID - TIC number, MIAP etc - start with this, then make Supplier ID
# Examples: TIC2040-001; MOZ42-001; SAF43-001; MIAP_207(B/AIA/ISP); CSF_1ABC_20; 05-008*#5; OB4-REP3_026; AAP_14; SU-Fungi-A16
original_df.insert(8, 'sourceID', original_df['year'].astype(str))
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,species,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,2048,wrong,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,2042,next,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,2041,board,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,2042,themselves,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,2041,though,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,2049,chair,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,2042,catch,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,2048,anything,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,2047,probably,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


In [342]:
numTIC = len(original_df[original_df.year == 2041])
numMOZ = len(original_df[original_df.year == 2042])
numSAF = len(original_df[original_df.year == 2043])
numMIAP = len(original_df[original_df.year == 2044])
numCSF = len(original_df[original_df.year == 2045])
numStrain = len(original_df[original_df.year == 2046])
numOB = len(original_df[original_df.year == 2047])
numAAP = len(original_df[original_df.year == 2048])
numSU = len(original_df[original_df.year == 2049])

In [343]:
# Examples: TIC2040-001; AMRU; MOZ42-001; SAF43-001; MIAP_207(B/AIA/ISP); CSF_1ABC_20; 05-008*#5; OB4-REP3_026; AAP_14; SU-Fungi-A16
original_df.loc[original_df['year'] == 2041, 'sourceID'] = 'TIC' + original_df['sourceID']
original_df.loc[original_df['year'] == 2041, 'sourceID'] = original_df.loc[original_df['year'] == 2041, 'sourceID'].str.cat(map(str, range(1, 1 + numTIC)), sep='-')
original_df.loc[original_df['year'] == 2042, 'sourceID'] = 'MOZ42'
original_df.loc[original_df['year'] == 2042, 'sourceID'] = original_df.loc[original_df['year'] == 2042, 'sourceID'].str.cat(map(str, range(1, 1 + numMOZ)), sep='-')
original_df.loc[original_df['year'] == 2043, 'sourceID'] = 'SAF43'
original_df.loc[original_df['year'] == 2043, 'sourceID'] = original_df.loc[original_df['year'] == 2043, 'sourceID'].str.cat(map(str, range(1, 1 + numSAF)), sep='-')
original_df.loc[original_df['year'] == 2044, 'sourceID'] = 'MIAP'
original_df.loc[original_df['year'] == 2044, 'sourceID'] = original_df.loc[original_df['year'] == 2044, 'sourceID'].str.cat(map(str, range(1, 1 + numMIAP)), sep='_')
original_df.loc[original_df['year'] == 2045, 'sourceID'] = 'CSF_1ABC'
original_df.loc[original_df['year'] == 2045, 'sourceID'] = original_df.loc[original_df['year'] == 2045, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numCSF)) // 3 + 1), sep='_')
original_df.loc[original_df['year'] == 2046, 'sourceID'] = ''
original_df.loc[original_df['year'] == 2046, 'sourceID'] = original_df.loc[original_df['year'] == 2046, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numStrain)) // 900 + 1))
original_df.loc[original_df['year'] == 2046, 'sourceID'] = original_df.loc[original_df['year'] == 2046, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numStrain)) // 90 + 1), sep='-')
original_df.loc[original_df['year'] == 2046, 'sourceID'] = original_df.loc[original_df['year'] == 2046, 'sourceID'].str.cat(map(str, np.array(range(numStrain)) // 3 + 1), sep='*#')
original_df.loc[original_df['year'] == 2047, 'sourceID'] = 'OB4-REP'
original_df.loc[original_df['year'] == 2047, 'sourceID'] = original_df.loc[original_df['year'] == 2047, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numOB)) // 120 + 1))
original_df.loc[original_df['year'] == 2047, 'sourceID'] = original_df.loc[original_df['year'] == 2047, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numOB)) // 3 + 1), sep='_')
original_df.loc[original_df['year'] == 2048, 'sourceID'] = 'AAP'
original_df.loc[original_df['year'] == 2048, 'sourceID'] = original_df.loc[original_df['year'] == 2048, 'sourceID'].str.cat(map(str, range(1, 1 + numAAP)), sep='_')
original_df.loc[original_df['year'] == 2049, 'sourceID'] = 'SU-Fungi-A'
original_df.loc[original_df['year'] == 2049, 'sourceID'] = original_df.loc[original_df['year'] == 2049, 'sourceID'].str.cat(map(str, range(1, 1 + numSU)))
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,species,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,wrong,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,next,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,board,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,themselves,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,though,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,SU-Fungi-A10,chair,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,catch,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,AAP_18,anything,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,probably,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


Inserting Source type

In [344]:
# Source type - based on source ID (microbial vs sponge vs stromatolite)
# Examples: TIC2041-001 -- MNP; MOZ42-001 -- MNP; SAF43-001 -- MNP; MIAP_207(B/AIA/ISP) -- Marion Islands; CSF_1ABC_20 -- Stromatolites; 05-008*#5 -- Microbial; OB4-REP3_026 -- Stromatolites; AAP_14 -- ; SU-Fungi-A16 -- Soil
original_df.insert(9, 'sourceType', 'MNP')
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,sourceType,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,MNP,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,MNP,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,MNP,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,MNP,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,MNP,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,SU-Fungi-A10,MNP,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,MNP,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,AAP_18,MNP,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,MNP,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


In [345]:
# Examples: TIC2040-001 -- MNP; AMRU001 -- Microbial; MOZ42-001 -- MNP; SAF43-001 -- MNP; MIAP_207(B/AIA/ISP) -- Marion Islands; CSF_1ABC_20 -- Stromatolites; 05-008*#5 -- Microbial; OB4-REP3_026 -- Stromatolites; AAP_14 -- ; SU-Fungi-A16 -- Soil
original_df.loc[original_df['year'] == 2044, 'sourceType'] = 'Marion Islands'
original_df.loc[original_df['year'] == 2045, 'sourceType'] = 'Stromatolites'
original_df.loc[original_df['year'] == 2046, 'sourceType'] = 'Microbial'
original_df.loc[original_df['year'] == 2047, 'sourceType'] = 'Stromatolites'
original_df.loc[original_df['year'] == 2048, 'sourceType'] = 'Bacterial'
original_df.loc[original_df['year'] == 2049, 'sourceType'] = 'Soil'
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,sourceType,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,Bacterial,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,MNP,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,MNP,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,MNP,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,MNP,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,SU-Fungi-A10,Soil,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,MNP,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,AAP_18,Bacterial,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,Stromatolites,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


Inserting SupplierID

In [346]:
# Supplier ID - based on date of collection, so must be same or earlier year than submittedDate
original_df.insert(1, 'supplierID', original_df['sourceID'].astype(str))
original_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,MOZ42-1,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,TIC2041-1,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,MOZ42-2,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,TIC2041-2,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,SU-Fungi-A10,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,SU-Fungi-A10,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,MOZ42-17,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,AAP_18,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,AAP_18,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,OB4-REP1_3,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


In [347]:
# Examples: TIC2040-001; MOZ42-001; SAF43-001; MIAP_207(B/AIA/ISP); CSF_1ABC_20; 05-008*#5; OB4-REP3_026; AAP_14; SU-Fungi-A16
# To: TIC2040-001JK; MOZ42-001JK; SAF42-001JK ; MIAP_207(B/AIA/ISP) keep same; AMRU077_A; R1 - R3; AMRU026_A; AAP_14; SU-Fungi-A16

# original_df.loc[original_df['year'] == 2045, 'supplierID'] = original_df.loc[original_df['year'] == 2045, 'supplierID'].str.cat(map(chr, (np.array(range(1, 1 + numCSF)) % 3) + 65))
original_df.loc[original_df['year'] == 2045, 'supplierID'] = 'AMRU'
original_df.loc[original_df['year'] == 2045, 'supplierID'] = original_df.loc[original_df['year'] == 2045, 'supplierID'].str.cat(map(str, ((np.array(range(1, 1 + numCSF)) // 3 + 1) + (np.array(range(1, 1 + numCSF)) // 3))))
original_df.loc[original_df['year'] == 2045, 'supplierID'] = original_df.loc[original_df['year'] == 2045, 'supplierID'].str.cat(map(chr, (np.array(range(1, 1 + numCSF)) % 3) + 65))

original_df.loc[original_df['year'] == 2047, 'supplierID'] = 'AMRU'
original_df.loc[original_df['year'] == 2047, 'supplierID'] = original_df.loc[original_df['year'] == 2047, 'supplierID'].str.cat(map(str, ((np.array(range(1, 1 + numOB)) // 3 + 1) + (np.array(range(1, 1 + numOB)) // 3 + 1))))
original_df.loc[original_df['year'] == 2047, 'supplierID'] = original_df.loc[original_df['year'] == 2047, 'supplierID'].str.cat(map(chr, (np.array(range(1, 1 + numOB)) % 3) + 65))

original_df.loc[original_df['year'] == 2046, 'supplierID'] = original_df.loc[original_df['year'] == 2046, 'supplierID'] + 'R'
original_df.loc[original_df['year'] == 2046, 'supplierID'] = original_df.loc[original_df['year'] == 2046, 'supplierID'].str.cat(map(str, (np.array(range(numStrain)) % 3 + 1)))

original_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol
1,AMR00002,MOZ42-1,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
2,AMR00003,TIC2041-1,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
3,AMR00004,MOZ42-2,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,...,Preston,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
4,AMR00005,TIC2041-2,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,SU-Fungi-A10,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,SU-Fungi-A10,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
96,AMR00097,MOZ42-17,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol
97,AMR00098,AAP_18,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,AAP_18,...,Preston,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol
98,AMR00099,AMRU6A,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol


In [348]:
# Making column with initials of submitter
nameSeries = original_df['submittedBy']
initialList = []
for index, value in nameSeries.items():
    initial = ""
    allNames = value.split(" ")
    for name in allNames:
        initial = initial+name[0].upper()
    # print(initial)
    initialList.append(initial)

initialSeries = pd.Series(initialList)
print(initialSeries)


0     JR
1     JR
2     RH
3     JR
4     JR
      ..
95    RH
96    RH
97    RH
98    RH
99    JR
Length: 100, dtype: object


In [349]:
# Add initials column and initials to correct supplier ID types
original_df.insert(26, 'submitterInitials', initialSeries)
original_df.loc[original_df['year'] == 2041, 'supplierID'] = original_df.loc[original_df['year'] == 2041, 'supplierID'].astype(str) + original_df.loc[original_df['year'] == 2041, 'submitterInitials'].astype(str)
original_df.loc[original_df['year'] == 2042, 'supplierID'] = original_df.loc[original_df['year'] == 2042, 'supplierID'].astype(str) + original_df.loc[original_df['year'] == 2042, 'submitterInitials'].astype(str)
original_df.loc[original_df['year'] == 2043, 'supplierID'] = original_df.loc[original_df['year'] == 2043, 'supplierID'].astype(str) + original_df.loc[original_df['year'] == 2043, 'submitterInitials'].astype(str)
original_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,submitterInitials
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol,JR
1,AMR00002,MOZ42-1JR,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,JR
2,AMR00003,TIC2041-1RH,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,...,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO,RH
3,AMR00004,MOZ42-2JR,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO,JR
4,AMR00005,TIC2041-2JR,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO,JR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AMR00096,SU-Fungi-A10,2049,Joshua Keener,27/01/2051,Rhodes MS/NMR,20/01/2051,75.5,N/A,SU-Fungi-A10,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol,RH
96,AMR00097,MOZ42-17RH,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol,RH
97,AMR00098,AAP_18,2048,Joshua Keener,24/07/2053,Rhodes LC/MS + NMR & Bioassay,17/07/2053,82.8,N/A,AAP_18,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,Methanol,RH
98,AMR00099,AMRU6A,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol,RH


Fractions: MIAP, SU-Fungi and AAP records are never fractionated. Therefore, we must remove the records with years 2044, 2048 and 2049

In [350]:
subset_df = original_df.copy(deep=True)
indexFraction = subset_df[(subset_df['year'] == 2044) | (subset_df['year'] == 2048) | (subset_df['year'] == 2049)].index
subset_df.drop(indexFraction, inplace=True)
subset_df.reset_index(drop=True, inplace=True)
subset_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,submitterInitials
0,AMR00002,MOZ42-1JR,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,JR
1,AMR00003,TIC2041-1RH,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,...,Robert Hobson,N/A,N/A,N/A,N/A,N,N,Y,DMSO,RH
2,AMR00004,MOZ42-2JR,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO,JR
3,AMR00005,TIC2041-2JR,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,Y,DMSO,JR
4,AMR00006,MOZ42-3RH,2042,Ruby Johnson,26/04/2050,Rhodes LC/MS + NMR & Bioassay,19/04/2050,53.2,N/A,MOZ42-3,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,Y,DMSO,RH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,AMR00094,AMRU4C,2047,Ruby Johnson,14/07/2052,Rhodes Bioassay,07/07/2052,84.7,N/A,OB4-REP1_2,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,RJ
63,AMR00095,MOZ42-16RJ,2042,Joshua Keener,11/05/2052,Membrane Disruption assay,04/05/2052,53.0,N/A,MOZ42-16,...,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice3,DMSO,RJ
64,AMR00097,MOZ42-17RH,2042,Ruby Johnson,07/08/2051,Rhodes Bioassay/Rhodes MS,31/07/2051,58.7,N/A,MOZ42-17,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,N,Methanol,RH
65,AMR00099,AMRU6A,2047,Joshua Keener,23/09/2053,Rhodes MS/NMR,16/09/2053,51.4,N/A,OB4-REP1_3,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,Methanol,RH


MNP Spreadsheet

MNP ID -> process

Supplier ID -> fraction + process

Assigned by -> process

Date assigned -> process

Destination -> process

Date submitted -> fraction

Amount available -> fraction

Sample type -> fraction

Concentration -> fraction

Solvent -> fraction

Project -> fraction

Source of sample -> fraction (from extract/fraction table)

Species -> removed

Permit no -> removed

Research group -> fraction

Submitted by -> fraction

Comments -> removed

Level (1st, 2nd, 3rd level etc) -> fraction

In [351]:
print(list(subset_df.columns.values))

['amrID', 'supplierID', 'year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'amountAvailable', 'synthesisPotential', 'sourceID', 'sourceType', 'species', 'permitNo', 'pure', 'ms', 'nmr', 'researchGroup', 'submittedBy', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature', 'solvent', 'submitterInitials']


In [352]:
# Need to remove columns that only extracts had/ must be replaced
# List: AMR ID, Source Id replaced with Supplier ID, source type, Destination, supplied date, synthesis potential, pure, MS, NMR, researchGroup
# solubility, stereoComments, SMILE, mw, additional metadata, existing patent, existing literature
# Will add 21 days to assigned date to get supplied date, and then add another week to that to get assigned date
subset_df = subset_df[["supplierID", "year", "assignedDate", "species", "permitNo"]]
subset_df

,supplierID,year,assignedDate,species,permitNo
0,MOZ42-1JR,2042,06/01/2051,next,RES2042
1,TIC2041-1RH,2041,31/01/2050,board,RES2041
2,MOZ42-2JR,2042,16/10/2050,themselves,RES2042
3,TIC2041-2JR,2041,21/12/2054,though,RES2041
4,MOZ42-3RH,2042,26/04/2050,memory,RES2042
...,...,...,...,...,...
62,AMRU4C,2047,14/07/2052,marriage,RES2047
63,MOZ42-16RJ,2042,11/05/2052,goal,RES2042
64,MOZ42-17RH,2042,07/08/2051,catch,RES2042
65,AMRU6A,2047,23/09/2053,probably,RES2047


In [353]:
# Date assigned - do a week after submittedDate
subset_df.assignedDate = pd.to_datetime(subset_df.assignedDate, format='%d/%m/%Y')
subset_df.insert(3, 'submittedDate', subset_df['assignedDate'])
subset_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\1737996625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.assignedDate = pd.to_datetime(subset_df.assignedDate, format='%d/%m/%Y')


,supplierID,year,assignedDate,submittedDate,species,permitNo
0,MOZ42-1JR,2042,2051-01-06,2051-01-06,next,RES2042
1,TIC2041-1RH,2041,2050-01-31,2050-01-31,board,RES2041
2,MOZ42-2JR,2042,2050-10-16,2050-10-16,themselves,RES2042
3,TIC2041-2JR,2041,2054-12-21,2054-12-21,though,RES2041
4,MOZ42-3RH,2042,2050-04-26,2050-04-26,memory,RES2042
...,...,...,...,...,...,...
62,AMRU4C,2047,2052-07-14,2052-07-14,marriage,RES2047
63,MOZ42-16RJ,2042,2052-05-11,2052-05-11,goal,RES2042
64,MOZ42-17RH,2042,2051-08-07,2051-08-07,catch,RES2042
65,AMRU6A,2047,2053-09-23,2053-09-23,probably,RES2047


In [354]:
subset_df['submittedDate'] = subset_df['submittedDate'] + timedelta(days=21)
subset_df['submittedDate'] = subset_df['submittedDate'].dt.strftime('%d/%m/%Y')
subset_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\1821189252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['submittedDate'] = subset_df['submittedDate'] + timedelta(days=21)
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\1821189252.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['submittedDate'] = subset_df['submittedDate'].dt.strftime('%d/%m/%Y')


,supplierID,year,assignedDate,submittedDate,species,permitNo
0,MOZ42-1JR,2042,2051-01-06,27/01/2051,next,RES2042
1,TIC2041-1RH,2041,2050-01-31,21/02/2050,board,RES2041
2,MOZ42-2JR,2042,2050-10-16,06/11/2050,themselves,RES2042
3,TIC2041-2JR,2041,2054-12-21,11/01/2055,though,RES2041
4,MOZ42-3RH,2042,2050-04-26,17/05/2050,memory,RES2042
...,...,...,...,...,...,...
62,AMRU4C,2047,2052-07-14,04/08/2052,marriage,RES2047
63,MOZ42-16RJ,2042,2052-05-11,01/06/2052,goal,RES2042
64,MOZ42-17RH,2042,2051-08-07,28/08/2051,catch,RES2042
65,AMRU6A,2047,2053-09-23,14/10/2053,probably,RES2047


In [355]:
# SubmittedDate is three weeks after extract assignedDate

# AssignedDate is one week after submittedDate
subset_df['assignedDate'] = subset_df['assignedDate'] + timedelta(days=28)
subset_df['assignedDate'] = subset_df['assignedDate'].dt.strftime('%d/%m/%Y')
subset_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2221413924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['assignedDate'] = subset_df['assignedDate'] + timedelta(days=28)
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2221413924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['assignedDate'] = subset_df['assignedDate'].dt.strftime('%d/%m/%Y')


,supplierID,year,assignedDate,submittedDate,species,permitNo
0,MOZ42-1JR,2042,03/02/2051,27/01/2051,next,RES2042
1,TIC2041-1RH,2041,28/02/2050,21/02/2050,board,RES2041
2,MOZ42-2JR,2042,13/11/2050,06/11/2050,themselves,RES2042
3,TIC2041-2JR,2041,18/01/2055,11/01/2055,though,RES2041
4,MOZ42-3RH,2042,24/05/2050,17/05/2050,memory,RES2042
...,...,...,...,...,...,...
62,AMRU4C,2047,11/08/2052,04/08/2052,marriage,RES2047
63,MOZ42-16RJ,2042,08/06/2052,01/06/2052,goal,RES2042
64,MOZ42-17RH,2042,04/09/2051,28/08/2051,catch,RES2042
65,AMRU6A,2047,21/10/2053,14/10/2053,probably,RES2047


In [356]:
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    binaryChoice = ['Y', 'N']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound X in fractionation', 'Used compound Y in fractionation', None, None, '50:50 setting', '20:20 setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_MNR_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

fractionSubset_df = create_MNR_data(fake, len(subset_df))
fractionSubset_df


,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Joshua Keener,RU Bioassay,SPE fraction,Smith,Priscilla Evans,DCM/MeOH,0.25,20:20 setting
1,Ruby Johnson,RU LC/MS,Fraction from flash,Roodt,Amanda Bowker,DCM:MeOH 2:1,0.50,20:20 setting
2,Ruby Johnson,RU LC-MS & 1H-NMR,Fraction,Roodt,Ruby Johnson,MeOH,1.00,50:50 setting
3,Ruby Johnson,CD Experiment,Fraction from flash,Smith,Priscilla Evans,DCM/MeOH,0.50,None
4,Ruby Johnson,RU LC-MS & 1H-NMR,SPE fraction,Roodt,Priscilla Evans,DCM/MeOH,50.00,None
...,...,...,...,...,...,...,...,...
62,Joshua Keener,CD Experiment,SPE fraction,Roodt,Amanda Bowker,DCM/MeOH,20.00,None
63,Ruby Johnson,Rhodes MS,Fraction from flash,Smith,Ruby Johnson,MeOH,0.10,Used compound X in fractionation
64,Joshua Keener,RU LC/MS,Fraction from flash,Roodt,Amanda Bowker,DCM:MeOH 2:1,0.10,None
65,Ruby Johnson,RU LC-MS & 1H-NMR,Fraction from flash,Smith,Ruby Johnson,MeOH,0.10,Used compound X in fractionation


In [357]:
# Join subsets together to get almost full MNP table
newFraction_df = pd.concat([subset_df, fractionSubset_df], axis=1)
newFraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newFraction_df = newFraction_df[orderCols]
newFraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,03/02/2051,RU Bioassay,27/01/2051,SPE fraction,0.25,DCM/MeOH,MOZ42-1JR,next,RES2042,Smith,Priscilla Evans,20:20 setting
1,2041,Ruby Johnson,28/02/2050,RU LC/MS,21/02/2050,Fraction from flash,0.50,DCM:MeOH 2:1,TIC2041-1RH,board,RES2041,Roodt,Amanda Bowker,20:20 setting
2,2042,Ruby Johnson,13/11/2050,RU LC-MS & 1H-NMR,06/11/2050,Fraction,1.00,MeOH,MOZ42-2JR,themselves,RES2042,Roodt,Ruby Johnson,50:50 setting
3,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.50,DCM/MeOH,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,None
4,2042,Ruby Johnson,24/05/2050,RU LC-MS & 1H-NMR,17/05/2050,SPE fraction,50.00,DCM/MeOH,MOZ42-3RH,memory,RES2042,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2047,Joshua Keener,11/08/2052,CD Experiment,04/08/2052,SPE fraction,20.00,DCM/MeOH,AMRU4C,marriage,RES2047,Roodt,Amanda Bowker,None
63,2042,Ruby Johnson,08/06/2052,Rhodes MS,01/06/2052,Fraction from flash,0.10,MeOH,MOZ42-16RJ,goal,RES2042,Smith,Ruby Johnson,Used compound X in fractionation
64,2042,Joshua Keener,04/09/2051,RU LC/MS,28/08/2051,Fraction from flash,0.10,DCM:MeOH 2:1,MOZ42-17RH,catch,RES2042,Roodt,Amanda Bowker,None
65,2047,Ruby Johnson,21/10/2053,RU LC-MS & 1H-NMR,14/10/2053,Fraction from flash,0.10,MeOH,AMRU6A,probably,RES2047,Smith,Ruby Johnson,Used compound X in fractionation


In [358]:
# Get 20 records from subset that is fractionated
numSampled1 = 20
fraction1_df = newFraction_df.sample(n = numSampled1, random_state=1231, ignore_index=True)
fraction2_df = newFraction_df.sample(n = numSampled1, random_state=1232, ignore_index=True)
fraction3_df = newFraction_df.sample(n = numSampled1, random_state=1233, ignore_index=True)
fraction4_df = newFraction_df.sample(n = numSampled1, random_state=1234, ignore_index=True)
fraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.00,DCM/MeOH,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,None
1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.00,DCM/MeOH,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None
2,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.50,DCM/MeOH,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,None
3,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.00,DCM:MeOH 2:1,AMRU3B,day,RES2045,Smith,Priscilla Evans,50:50 setting
4,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.10,DCM/MeOH,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,20:20 setting
5,2042,Ruby Johnson,24/05/2050,RU LC-MS & 1H-NMR,17/05/2050,SPE fraction,50.00,DCM/MeOH,MOZ42-3RH,memory,RES2042,Roodt,Priscilla Evans,None
6,2041,Joshua Keener,19/04/2051,CD Experiment,12/04/2051,Fraction from flash,0.50,DCM:MeOH 2:1,TIC2041-12RJ,various,RES2041,Roodt,Priscilla Evans,50:50 setting
7,2042,Joshua Keener,03/02/2051,RU Bioassay,27/01/2051,SPE fraction,0.25,DCM/MeOH,MOZ42-1JR,next,RES2042,Smith,Priscilla Evans,20:20 setting
8,2046,Joshua Keener,01/09/2052,RU LC/MS,25/08/2052,HPLC Fraction,0.10,MeOH,1-1*#1R3,difficult,RES2046,Roodt,Ruby Johnson,50:50 setting
9,2042,Joshua Keener,12/05/2051,Rhodes MS,05/05/2051,SPE fraction,50.00,MeOH,MOZ42-5JR,scene,RES2042,Roodt,Priscilla Evans,Used compound X in fractionation


In [359]:
fractionFirst_df = pd.concat([fraction1_df, fraction2_df, fraction3_df, fraction4_df], ignore_index=True)
fractionFirst_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.0,DCM/MeOH,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,None
1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None
2,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.5,DCM/MeOH,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,None
3,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.0,DCM:MeOH 2:1,AMRU3B,day,RES2045,Smith,Priscilla Evans,50:50 setting
4,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.1,DCM/MeOH,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,20:20 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2043,Joshua Keener,13/02/2054,Rhodes MS,06/02/2054,SPE fraction,0.1,DCM/MeOH,SAF43-4RH,free,RES2043,Smith,Amanda Bowker,Used compound Y in fractionation
76,2042,Joshua Keener,04/09/2051,RU LC/MS,28/08/2051,Fraction from flash,0.1,DCM:MeOH 2:1,MOZ42-17RH,catch,RES2042,Roodt,Amanda Bowker,None
77,2043,Joshua Keener,15/11/2050,RU LC-MS & 1H-NMR,08/11/2050,Fraction,50.0,DCM:MeOH 2:1,SAF43-2RH,take,RES2043,Roodt,Amanda Bowker,Used compound X in fractionation
78,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None


In [360]:
# Project - microbial for AMNRU, MNP/Marine for rest
fractionFirst_df.insert(8, 'project', 'Marine')
fractionFirst_df.loc[fractionFirst_df['year'] == 2045, 'project'] = 'Microbial'
fractionFirst_df.loc[fractionFirst_df['year'] == 2047, 'project'] = 'Microbial'
fractionFirst_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.0,DCM/MeOH,Marine,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,None
1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None
2,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.5,DCM/MeOH,Marine,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,None
3,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU3B,day,RES2045,Smith,Priscilla Evans,50:50 setting
4,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.1,DCM/MeOH,Marine,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,20:20 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2043,Joshua Keener,13/02/2054,Rhodes MS,06/02/2054,SPE fraction,0.1,DCM/MeOH,Marine,SAF43-4RH,free,RES2043,Smith,Amanda Bowker,Used compound Y in fractionation
76,2042,Joshua Keener,04/09/2051,RU LC/MS,28/08/2051,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,MOZ42-17RH,catch,RES2042,Roodt,Amanda Bowker,None
77,2043,Joshua Keener,15/11/2050,RU LC-MS & 1H-NMR,08/11/2050,Fraction,50.0,DCM:MeOH 2:1,Marine,SAF43-2RH,take,RES2043,Roodt,Amanda Bowker,Used compound X in fractionation
78,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None


In [361]:
# Parts that are different per fraction:
# Amount available
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amount_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amount_df = create_amount_data(fake, 80)

fractionFirst_df = pd.concat([amount_df, fractionFirst_df], axis=1)
fractionFirst_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,96.6,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.0,DCM/MeOH,Marine,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,None
1,44.1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None
2,0.7,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.5,DCM/MeOH,Marine,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,None
3,91.1,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU3B,day,RES2045,Smith,Priscilla Evans,50:50 setting
4,93.9,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.1,DCM/MeOH,Marine,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,20:20 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,18.6,2043,Joshua Keener,13/02/2054,Rhodes MS,06/02/2054,SPE fraction,0.1,DCM/MeOH,Marine,SAF43-4RH,free,RES2043,Smith,Amanda Bowker,Used compound Y in fractionation
76,43.6,2042,Joshua Keener,04/09/2051,RU LC/MS,28/08/2051,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,MOZ42-17RH,catch,RES2042,Roodt,Amanda Bowker,None
77,11.9,2043,Joshua Keener,15/11/2050,RU LC-MS & 1H-NMR,08/11/2050,Fraction,50.0,DCM:MeOH 2:1,Marine,SAF43-2RH,take,RES2043,Roodt,Amanda Bowker,Used compound X in fractionation
78,53.0,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None


In [362]:
# Add MNP ID
fractionFirst_df.insert(0, 'mnpID', range(1, 1 + len(fractionFirst_df)))
fractionFirst_df['mnpID'] = 'MNP' + fractionFirst_df['mnpID'].astype(str).str.zfill(5)
fractionFirst_df

,mnpID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,MNP00001,96.6,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.0,DCM/MeOH,Marine,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,None
1,MNP00002,44.1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None
2,MNP00003,0.7,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.5,DCM/MeOH,Marine,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,None
3,MNP00004,91.1,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU3B,day,RES2045,Smith,Priscilla Evans,50:50 setting
4,MNP00005,93.9,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.1,DCM/MeOH,Marine,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,20:20 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MNP00076,18.6,2043,Joshua Keener,13/02/2054,Rhodes MS,06/02/2054,SPE fraction,0.1,DCM/MeOH,Marine,SAF43-4RH,free,RES2043,Smith,Amanda Bowker,Used compound Y in fractionation
76,MNP00077,43.6,2042,Joshua Keener,04/09/2051,RU LC/MS,28/08/2051,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,MOZ42-17RH,catch,RES2042,Roodt,Amanda Bowker,None
77,MNP00078,11.9,2043,Joshua Keener,15/11/2050,RU LC-MS & 1H-NMR,08/11/2050,Fraction,50.0,DCM:MeOH 2:1,Marine,SAF43-2RH,take,RES2043,Roodt,Amanda Bowker,Used compound X in fractionation
78,MNP00079,53.0,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,None


In [363]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
# 
sourceSeries = fractionFirst_df['sourceID']
listSupplier = []
supplier_freq = {}

for index, value in sourceSeries.items():
    if (value in supplier_freq):
        supplier_freq[value] += 1
    else:
        supplier_freq[value] = 1
    listSupplier.append(value + "-" + str(supplier_freq[value]).zfill(2))

supplierSeries = pd.Series(listSupplier)
#print(supplierSeries)

fractionFirst_df.insert(1, 'supplierID', supplierSeries)
# Include level (currently all 1)
fractionFirst_df.insert(17, 'level', 1)
fractionFirst_df

,mnpID,supplierID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00001,TIC2041-3RH-01,96.6,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.0,DCM/MeOH,Marine,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,1,None
1,MNP00002,TIC2041-8RJ-01,44.1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,1,None
2,MNP00003,TIC2041-2JR-01,0.7,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.5,DCM/MeOH,Marine,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,1,None
3,MNP00004,AMRU3B-01,91.1,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU3B,day,RES2045,Smith,Priscilla Evans,1,50:50 setting
4,MNP00005,SAF43-5RH-01,93.9,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.1,DCM/MeOH,Marine,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,1,20:20 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MNP00076,SAF43-4RH-01,18.6,2043,Joshua Keener,13/02/2054,Rhodes MS,06/02/2054,SPE fraction,0.1,DCM/MeOH,Marine,SAF43-4RH,free,RES2043,Smith,Amanda Bowker,1,Used compound Y in fractionation
76,MNP00077,MOZ42-17RH-01,43.6,2042,Joshua Keener,04/09/2051,RU LC/MS,28/08/2051,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,MOZ42-17RH,catch,RES2042,Roodt,Amanda Bowker,1,None
77,MNP00078,SAF43-2RH-02,11.9,2043,Joshua Keener,15/11/2050,RU LC-MS & 1H-NMR,08/11/2050,Fraction,50.0,DCM:MeOH 2:1,Marine,SAF43-2RH,take,RES2043,Roodt,Amanda Bowker,1,Used compound X in fractionation
78,MNP00079,TIC2041-8RJ-04,53.0,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.0,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,1,None


Subfractions

In [364]:
subfraction_df = fractionFirst_df.copy(deep=True)
subfraction_df = subfraction_df[["supplierID", "year", "assignedDate", "species", "permitNo", "project"]]
subfraction_df

,supplierID,year,assignedDate,species,permitNo,project
0,TIC2041-3RH-01,2041,10/02/2052,realize,RES2041,Marine
1,TIC2041-8RJ-01,2041,14/12/2053,discuss,RES2041,Marine
2,TIC2041-2JR-01,2041,18/01/2055,though,RES2041,Marine
3,AMRU3B-01,2045,15/01/2051,day,RES2045,Microbial
4,SAF43-5RH-01,2043,29/09/2050,attorney,RES2043,Marine
...,...,...,...,...,...,...
75,SAF43-4RH-01,2043,13/02/2054,free,RES2043,Marine
76,MOZ42-17RH-01,2042,04/09/2051,catch,RES2042,Marine
77,SAF43-2RH-02,2043,15/11/2050,take,RES2043,Marine
78,TIC2041-8RJ-04,2041,14/12/2053,discuss,RES2041,Marine


In [365]:
# Inserting assignedDate and submittedDate
subfraction_df.assignedDate = pd.to_datetime(subfraction_df.assignedDate, format='%d/%m/%Y')
subfraction_df.insert(3, 'submittedDate', subfraction_df['assignedDate'])
# Date submitted - do 3 weeks after previous assignedDate
subfraction_df['submittedDate'] = subfraction_df['submittedDate'] + timedelta(days=21)
subfraction_df['submittedDate'] = subfraction_df['submittedDate'].dt.strftime('%d/%m/%Y')
# Date assigned - do a week after submittedDate
subfraction_df['assignedDate'] = subfraction_df['assignedDate'] + timedelta(days=28)
subfraction_df['assignedDate'] = subfraction_df['assignedDate'].dt.strftime('%d/%m/%Y')
subfraction_df

,supplierID,year,assignedDate,submittedDate,species,permitNo,project
0,TIC2041-3RH-01,2041,09/03/2052,02/03/2052,realize,RES2041,Marine
1,TIC2041-8RJ-01,2041,11/01/2054,04/01/2054,discuss,RES2041,Marine
2,TIC2041-2JR-01,2041,15/02/2055,08/02/2055,though,RES2041,Marine
3,AMRU3B-01,2045,12/02/2051,05/02/2051,day,RES2045,Microbial
4,SAF43-5RH-01,2043,27/10/2050,20/10/2050,attorney,RES2043,Marine
...,...,...,...,...,...,...,...
75,SAF43-4RH-01,2043,13/03/2054,06/03/2054,free,RES2043,Marine
76,MOZ42-17RH-01,2042,02/10/2051,25/09/2051,catch,RES2042,Marine
77,SAF43-2RH-02,2043,13/12/2050,06/12/2050,take,RES2043,Marine
78,TIC2041-8RJ-04,2041,11/01/2054,04/01/2054,discuss,RES2041,Marine


In [366]:
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound A in fractionation', 'Used compound B in fractionation', None, None, '80:20 setting', '60:40 setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_subfraction_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

subfractionSubset_df = create_subfraction_data(fake, len(subfraction_df))
subfractionSubset_df

,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Joshua Keener,CD Experiment,SPE fraction,Roodt,Amanda Bowker,DCM:MeOH 2:1,1.00,80:20 setting
1,Ruby Johnson,RU Bioassay,HPLC Fraction,Roodt,Amanda Bowker,DCM:MeOH 2:1,50.00,60:40 setting
2,Ruby Johnson,Rhodes MS,Fraction from flash,Roodt,Priscilla Evans,DCM/MeOH,1.00,80:20 setting
3,Ruby Johnson,RU LC-MS & 1H-NMR,Fraction from flash,Smith,Priscilla Evans,MeOH,1.00,None
4,Ruby Johnson,RU LC-MS & 1H-NMR,HPLC Fraction,Roodt,Priscilla Evans,DCM:MeOH 2:1,0.10,Used compound B in fractionation
...,...,...,...,...,...,...,...,...
75,Joshua Keener,Rhodes MS,HPLC Fraction,Smith,Ruby Johnson,DCM/MeOH,1.00,80:20 setting
76,Ruby Johnson,RU Bioassay,Fraction,Roodt,Ruby Johnson,MeOH,0.10,60:40 setting
77,Joshua Keener,RU LC-MS & 1H-NMR,Fraction,Smith,Priscilla Evans,MeOH,50.00,Used compound A in fractionation
78,Joshua Keener,RU LC-MS & 1H-NMR,SPE fraction,Roodt,Amanda Bowker,MeOH,0.25,80:20 setting


In [367]:
# Join subsets together to get almost full MNP table
newSubFraction_df = pd.concat([subfraction_df, subfractionSubset_df], axis=1)
newSubFraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'project', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newSubFraction_df = newSubFraction_df[orderCols]
newSubFraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2041,Joshua Keener,09/03/2052,CD Experiment,02/03/2052,SPE fraction,1.00,DCM:MeOH 2:1,Marine,TIC2041-3RH-01,realize,RES2041,Roodt,Amanda Bowker,80:20 setting
1,2041,Ruby Johnson,11/01/2054,RU Bioassay,04/01/2054,HPLC Fraction,50.00,DCM:MeOH 2:1,Marine,TIC2041-8RJ-01,discuss,RES2041,Roodt,Amanda Bowker,60:40 setting
2,2041,Ruby Johnson,15/02/2055,Rhodes MS,08/02/2055,Fraction from flash,1.00,DCM/MeOH,Marine,TIC2041-2JR-01,though,RES2041,Roodt,Priscilla Evans,80:20 setting
3,2045,Ruby Johnson,12/02/2051,RU LC-MS & 1H-NMR,05/02/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU3B-01,day,RES2045,Smith,Priscilla Evans,None
4,2043,Ruby Johnson,27/10/2050,RU LC-MS & 1H-NMR,20/10/2050,HPLC Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-5RH-01,attorney,RES2043,Roodt,Priscilla Evans,Used compound B in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2043,Joshua Keener,13/03/2054,Rhodes MS,06/03/2054,HPLC Fraction,1.00,DCM/MeOH,Marine,SAF43-4RH-01,free,RES2043,Smith,Ruby Johnson,80:20 setting
76,2042,Ruby Johnson,02/10/2051,RU Bioassay,25/09/2051,Fraction,0.10,MeOH,Marine,MOZ42-17RH-01,catch,RES2042,Roodt,Ruby Johnson,60:40 setting
77,2043,Joshua Keener,13/12/2050,RU LC-MS & 1H-NMR,06/12/2050,Fraction,50.00,MeOH,Marine,SAF43-2RH-02,take,RES2043,Smith,Priscilla Evans,Used compound A in fractionation
78,2041,Joshua Keener,11/01/2054,RU LC-MS & 1H-NMR,04/01/2054,SPE fraction,0.25,MeOH,Marine,TIC2041-8RJ-04,discuss,RES2041,Roodt,Amanda Bowker,80:20 setting


In [368]:
# Get 20 records from subset that is fractionated
numSampled2 = 20
subfraction1_df = newSubFraction_df.sample(n = numSampled2, random_state=1231, ignore_index=True)
subfraction2_df = newSubFraction_df.sample(n = numSampled2, random_state=1232, ignore_index=True)
subfraction3_df = newSubFraction_df.sample(n = numSampled2, random_state=1233, ignore_index=True)
subfraction4_df = newSubFraction_df.sample(n = numSampled2, random_state=1234, ignore_index=True)
subfraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,19/04/2052,CD Experiment,12/04/2052,HPLC Fraction,50.00,MeOH,Marine,MOZ42-7JR-01,clear,RES2042,Roodt,Ruby Johnson,None
1,2046,Ruby Johnson,29/09/2052,RU LC-MS & 1H-NMR,22/09/2052,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,1-1*#1R3-01,difficult,RES2046,Roodt,Ruby Johnson,None
2,2045,Ruby Johnson,05/02/2055,RU LC/MS,29/01/2055,Fraction from flash,0.50,MeOH,Microbial,AMRU1B-01,likely,RES2045,Roodt,Amanda Bowker,80:20 setting
3,2047,Joshua Keener,18/11/2053,RU LC-MS & 1H-NMR,11/11/2053,SPE fraction,0.50,MeOH,Microbial,AMRU6A-02,probably,RES2047,Roodt,Amanda Bowker,Used compound B in fractionation
4,2046,Joshua Keener,18/12/2054,RU LC-MS & 1H-NMR,11/12/2054,HPLC Fraction,0.10,DCM/MeOH,Marine,1-1*#3R1-01,know,RES2046,Roodt,Priscilla Evans,None
5,2043,Ruby Johnson,27/10/2050,RU LC-MS & 1H-NMR,20/10/2050,HPLC Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-5RH-01,attorney,RES2043,Roodt,Priscilla Evans,Used compound B in fractionation
6,2045,Ruby Johnson,12/02/2051,RU LC-MS & 1H-NMR,05/02/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU3B-01,day,RES2045,Smith,Priscilla Evans,None
7,2042,Joshua Keener,03/03/2051,RU Bioassay,24/02/2051,Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01,next,RES2042,Smith,Ruby Johnson,60:40 setting
8,2042,Ruby Johnson,04/03/2055,CD Experiment,25/02/2055,HPLC Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-13RH-01,pay,RES2042,Smith,Priscilla Evans,None
9,2043,Joshua Keener,13/03/2054,Rhodes MS,06/03/2054,HPLC Fraction,1.00,DCM/MeOH,Marine,SAF43-4RH-01,free,RES2043,Smith,Ruby Johnson,80:20 setting


In [369]:
fractionSecond_df = pd.concat([subfraction1_df, subfraction2_df, subfraction3_df, subfraction4_df], ignore_index=True)
fractionSecond_df
# Already have project

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,19/04/2052,CD Experiment,12/04/2052,HPLC Fraction,50.00,MeOH,Marine,MOZ42-7JR-01,clear,RES2042,Roodt,Ruby Johnson,None
1,2046,Ruby Johnson,29/09/2052,RU LC-MS & 1H-NMR,22/09/2052,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,1-1*#1R3-01,difficult,RES2046,Roodt,Ruby Johnson,None
2,2045,Ruby Johnson,05/02/2055,RU LC/MS,29/01/2055,Fraction from flash,0.50,MeOH,Microbial,AMRU1B-01,likely,RES2045,Roodt,Amanda Bowker,80:20 setting
3,2047,Joshua Keener,18/11/2053,RU LC-MS & 1H-NMR,11/11/2053,SPE fraction,0.50,MeOH,Microbial,AMRU6A-02,probably,RES2047,Roodt,Amanda Bowker,Used compound B in fractionation
4,2046,Joshua Keener,18/12/2054,RU LC-MS & 1H-NMR,11/12/2054,HPLC Fraction,0.10,DCM/MeOH,Marine,1-1*#3R1-01,know,RES2046,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2041,Joshua Keener,17/05/2051,Rhodes MS,10/05/2051,SPE fraction,20.00,DCM/MeOH,Marine,TIC2041-12RJ-03,various,RES2041,Roodt,Priscilla Evans,None
76,2042,Joshua Keener,04/10/2050,RU Bioassay,27/09/2050,SPE fraction,0.25,MeOH,Marine,MOZ42-9RJ-02,game,RES2042,Smith,Ruby Johnson,None
77,2047,Joshua Keener,28/06/2051,RU LC-MS & 1H-NMR,21/06/2051,SPE fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU4A-01,myself,RES2047,Roodt,Ruby Johnson,None
78,2046,Joshua Keener,10/10/2050,Rhodes MS,03/10/2050,Fraction from flash,20.00,MeOH,Marine,1-1*#2R1-02,structure,RES2046,Smith,Priscilla Evans,None


In [370]:
# Parts that are different per subfraction:
# Amount available
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amountSub_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amountSub_df = create_amountSub_data(fake, 80)

fractionSecond_df = pd.concat([amountSub_df, fractionSecond_df], axis=1)
fractionSecond_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,31.4,2042,Joshua Keener,19/04/2052,CD Experiment,12/04/2052,HPLC Fraction,50.00,MeOH,Marine,MOZ42-7JR-01,clear,RES2042,Roodt,Ruby Johnson,None
1,50.9,2046,Ruby Johnson,29/09/2052,RU LC-MS & 1H-NMR,22/09/2052,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,1-1*#1R3-01,difficult,RES2046,Roodt,Ruby Johnson,None
2,18.5,2045,Ruby Johnson,05/02/2055,RU LC/MS,29/01/2055,Fraction from flash,0.50,MeOH,Microbial,AMRU1B-01,likely,RES2045,Roodt,Amanda Bowker,80:20 setting
3,28.2,2047,Joshua Keener,18/11/2053,RU LC-MS & 1H-NMR,11/11/2053,SPE fraction,0.50,MeOH,Microbial,AMRU6A-02,probably,RES2047,Roodt,Amanda Bowker,Used compound B in fractionation
4,67.0,2046,Joshua Keener,18/12/2054,RU LC-MS & 1H-NMR,11/12/2054,HPLC Fraction,0.10,DCM/MeOH,Marine,1-1*#3R1-01,know,RES2046,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,26.0,2041,Joshua Keener,17/05/2051,Rhodes MS,10/05/2051,SPE fraction,20.00,DCM/MeOH,Marine,TIC2041-12RJ-03,various,RES2041,Roodt,Priscilla Evans,None
76,85.7,2042,Joshua Keener,04/10/2050,RU Bioassay,27/09/2050,SPE fraction,0.25,MeOH,Marine,MOZ42-9RJ-02,game,RES2042,Smith,Ruby Johnson,None
77,10.1,2047,Joshua Keener,28/06/2051,RU LC-MS & 1H-NMR,21/06/2051,SPE fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU4A-01,myself,RES2047,Roodt,Ruby Johnson,None
78,14.4,2046,Joshua Keener,10/10/2050,Rhodes MS,03/10/2050,Fraction from flash,20.00,MeOH,Marine,1-1*#2R1-02,structure,RES2046,Smith,Priscilla Evans,None


In [371]:
fractionSecond_df.insert(0, 'mnpID', range(len(fractionFirst_df) + 1, 1 + len(fractionFirst_df) + len(fractionSecond_df)))
fractionSecond_df['mnpID'] = 'MNP' + fractionSecond_df['mnpID'].astype(str).str.zfill(5)
fractionSecond_df

,mnpID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,MNP00081,31.4,2042,Joshua Keener,19/04/2052,CD Experiment,12/04/2052,HPLC Fraction,50.00,MeOH,Marine,MOZ42-7JR-01,clear,RES2042,Roodt,Ruby Johnson,None
1,MNP00082,50.9,2046,Ruby Johnson,29/09/2052,RU LC-MS & 1H-NMR,22/09/2052,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,1-1*#1R3-01,difficult,RES2046,Roodt,Ruby Johnson,None
2,MNP00083,18.5,2045,Ruby Johnson,05/02/2055,RU LC/MS,29/01/2055,Fraction from flash,0.50,MeOH,Microbial,AMRU1B-01,likely,RES2045,Roodt,Amanda Bowker,80:20 setting
3,MNP00084,28.2,2047,Joshua Keener,18/11/2053,RU LC-MS & 1H-NMR,11/11/2053,SPE fraction,0.50,MeOH,Microbial,AMRU6A-02,probably,RES2047,Roodt,Amanda Bowker,Used compound B in fractionation
4,MNP00085,67.0,2046,Joshua Keener,18/12/2054,RU LC-MS & 1H-NMR,11/12/2054,HPLC Fraction,0.10,DCM/MeOH,Marine,1-1*#3R1-01,know,RES2046,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MNP00156,26.0,2041,Joshua Keener,17/05/2051,Rhodes MS,10/05/2051,SPE fraction,20.00,DCM/MeOH,Marine,TIC2041-12RJ-03,various,RES2041,Roodt,Priscilla Evans,None
76,MNP00157,85.7,2042,Joshua Keener,04/10/2050,RU Bioassay,27/09/2050,SPE fraction,0.25,MeOH,Marine,MOZ42-9RJ-02,game,RES2042,Smith,Ruby Johnson,None
77,MNP00158,10.1,2047,Joshua Keener,28/06/2051,RU LC-MS & 1H-NMR,21/06/2051,SPE fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU4A-01,myself,RES2047,Roodt,Ruby Johnson,None
78,MNP00159,14.4,2046,Joshua Keener,10/10/2050,Rhodes MS,03/10/2050,Fraction from flash,20.00,MeOH,Marine,1-1*#2R1-02,structure,RES2046,Smith,Priscilla Evans,None


In [372]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
sourceSubSeries = fractionSecond_df['sourceID']
listSubSupplier = []
subsupplier_freq = {}

for index, value in sourceSubSeries.items():
    if (value in subsupplier_freq):
        subsupplier_freq[value] += 1
    else:
        subsupplier_freq[value] = 1
    listSubSupplier.append(value + str(chr(subsupplier_freq[value] + 96)))

subsupplierSeries = pd.Series(listSubSupplier)
#print(supplierSeries)

fractionSecond_df.insert(1, 'supplierID', subsupplierSeries)
# Include level (2)
fractionSecond_df.insert(17, 'level', 2)
fractionSecond_df

,mnpID,supplierID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00081,MOZ42-7JR-01a,31.4,2042,Joshua Keener,19/04/2052,CD Experiment,12/04/2052,HPLC Fraction,50.00,MeOH,Marine,MOZ42-7JR-01,clear,RES2042,Roodt,Ruby Johnson,2,None
1,MNP00082,1-1*#1R3-01a,50.9,2046,Ruby Johnson,29/09/2052,RU LC-MS & 1H-NMR,22/09/2052,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,1-1*#1R3-01,difficult,RES2046,Roodt,Ruby Johnson,2,None
2,MNP00083,AMRU1B-01a,18.5,2045,Ruby Johnson,05/02/2055,RU LC/MS,29/01/2055,Fraction from flash,0.50,MeOH,Microbial,AMRU1B-01,likely,RES2045,Roodt,Amanda Bowker,2,80:20 setting
3,MNP00084,AMRU6A-02a,28.2,2047,Joshua Keener,18/11/2053,RU LC-MS & 1H-NMR,11/11/2053,SPE fraction,0.50,MeOH,Microbial,AMRU6A-02,probably,RES2047,Roodt,Amanda Bowker,2,Used compound B in fractionation
4,MNP00085,1-1*#3R1-01a,67.0,2046,Joshua Keener,18/12/2054,RU LC-MS & 1H-NMR,11/12/2054,HPLC Fraction,0.10,DCM/MeOH,Marine,1-1*#3R1-01,know,RES2046,Roodt,Priscilla Evans,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MNP00156,TIC2041-12RJ-03a,26.0,2041,Joshua Keener,17/05/2051,Rhodes MS,10/05/2051,SPE fraction,20.00,DCM/MeOH,Marine,TIC2041-12RJ-03,various,RES2041,Roodt,Priscilla Evans,2,None
76,MNP00157,MOZ42-9RJ-02a,85.7,2042,Joshua Keener,04/10/2050,RU Bioassay,27/09/2050,SPE fraction,0.25,MeOH,Marine,MOZ42-9RJ-02,game,RES2042,Smith,Ruby Johnson,2,None
77,MNP00158,AMRU4A-01d,10.1,2047,Joshua Keener,28/06/2051,RU LC-MS & 1H-NMR,21/06/2051,SPE fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU4A-01,myself,RES2047,Roodt,Ruby Johnson,2,None
78,MNP00159,1-1*#2R1-02a,14.4,2046,Joshua Keener,10/10/2050,Rhodes MS,03/10/2050,Fraction from flash,20.00,MeOH,Marine,1-1*#2R1-02,structure,RES2046,Smith,Priscilla Evans,2,None


Subsubfractions

In [373]:
subsubfraction_df = fractionSecond_df.copy(deep=True)
subsubfraction_df = subsubfraction_df[["supplierID", "year", "assignedDate", "species", "permitNo", "project"]]
subsubfraction_df

,supplierID,year,assignedDate,species,permitNo,project
0,MOZ42-7JR-01a,2042,19/04/2052,clear,RES2042,Marine
1,1-1*#1R3-01a,2046,29/09/2052,difficult,RES2046,Marine
2,AMRU1B-01a,2045,05/02/2055,likely,RES2045,Microbial
3,AMRU6A-02a,2047,18/11/2053,probably,RES2047,Microbial
4,1-1*#3R1-01a,2046,18/12/2054,know,RES2046,Marine
...,...,...,...,...,...,...
75,TIC2041-12RJ-03a,2041,17/05/2051,various,RES2041,Marine
76,MOZ42-9RJ-02a,2042,04/10/2050,game,RES2042,Marine
77,AMRU4A-01d,2047,28/06/2051,myself,RES2047,Microbial
78,1-1*#2R1-02a,2046,10/10/2050,structure,RES2046,Marine


In [374]:
# Inserting assignedDate and submittedDate
subsubfraction_df.assignedDate = pd.to_datetime(subsubfraction_df.assignedDate, format='%d/%m/%Y')
subsubfraction_df.insert(3, 'submittedDate', subsubfraction_df['assignedDate'])
# Date submitted - do 3 weeks after previous assignedDate
subsubfraction_df['submittedDate'] = subsubfraction_df['submittedDate'] + timedelta(days=21)
subsubfraction_df['submittedDate'] = subsubfraction_df['submittedDate'].dt.strftime('%d/%m/%Y')
# Date assigned - do a week after submittedDate
subsubfraction_df['assignedDate'] = subsubfraction_df['assignedDate'] + timedelta(days=28)
subsubfraction_df['assignedDate'] = subsubfraction_df['assignedDate'].dt.strftime('%d/%m/%Y')
subsubfraction_df

,supplierID,year,assignedDate,submittedDate,species,permitNo,project
0,MOZ42-7JR-01a,2042,17/05/2052,10/05/2052,clear,RES2042,Marine
1,1-1*#1R3-01a,2046,27/10/2052,20/10/2052,difficult,RES2046,Marine
2,AMRU1B-01a,2045,05/03/2055,26/02/2055,likely,RES2045,Microbial
3,AMRU6A-02a,2047,16/12/2053,09/12/2053,probably,RES2047,Microbial
4,1-1*#3R1-01a,2046,15/01/2055,08/01/2055,know,RES2046,Marine
...,...,...,...,...,...,...,...
75,TIC2041-12RJ-03a,2041,14/06/2051,07/06/2051,various,RES2041,Marine
76,MOZ42-9RJ-02a,2042,01/11/2050,25/10/2050,game,RES2042,Marine
77,AMRU4A-01d,2047,26/07/2051,19/07/2051,myself,RES2047,Microbial
78,1-1*#2R1-02a,2046,07/11/2050,31/10/2050,structure,RES2046,Marine


In [375]:
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound F in fractionation', 'Used compound G in fractionation', None, None, '70:30 setting', '20:80 setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_subsubfraction_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

subsubfractionSubset_df = create_subsubfraction_data(fake, len(subsubfraction_df))
subsubfractionSubset_df

,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Joshua Keener,RU LC/MS,Fraction from flash,Roodt,Ruby Johnson,DCM:MeOH 2:1,50.00,None
1,Joshua Keener,RU LC/MS,SPE fraction,Smith,Ruby Johnson,DCM:MeOH 2:1,0.25,None
2,Joshua Keener,RU LC-MS & 1H-NMR,SPE fraction,Smith,Ruby Johnson,MeOH,50.00,Used compound F in fractionation
3,Ruby Johnson,CD Experiment,SPE fraction,Roodt,Ruby Johnson,DCM/MeOH,20.00,70:30 setting
4,Joshua Keener,CD Experiment,Fraction,Roodt,Ruby Johnson,DCM/MeOH,1.00,None
...,...,...,...,...,...,...,...,...
75,Ruby Johnson,CD Experiment,Fraction from flash,Roodt,Ruby Johnson,MeOH,0.50,Used compound G in fractionation
76,Ruby Johnson,RU LC/MS,SPE fraction,Smith,Priscilla Evans,DCM/MeOH,0.25,None
77,Ruby Johnson,RU LC/MS,Fraction,Smith,Ruby Johnson,MeOH,0.10,Used compound G in fractionation
78,Joshua Keener,RU LC-MS & 1H-NMR,HPLC Fraction,Smith,Ruby Johnson,DCM:MeOH 2:1,0.50,None


In [376]:
newSubSubFraction_df = pd.concat([subsubfraction_df, subsubfractionSubset_df], axis=1)
newSubSubFraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'project', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newSubSubFraction_df = newSubSubFraction_df[orderCols]
newSubSubFraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,17/05/2052,RU LC/MS,10/05/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,MOZ42-7JR-01a,clear,RES2042,Roodt,Ruby Johnson,None
1,2046,Joshua Keener,27/10/2052,RU LC/MS,20/10/2052,SPE fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01a,difficult,RES2046,Smith,Ruby Johnson,None
2,2045,Joshua Keener,05/03/2055,RU LC-MS & 1H-NMR,26/02/2055,SPE fraction,50.00,MeOH,Microbial,AMRU1B-01a,likely,RES2045,Smith,Ruby Johnson,Used compound F in fractionation
3,2047,Ruby Johnson,16/12/2053,CD Experiment,09/12/2053,SPE fraction,20.00,DCM/MeOH,Microbial,AMRU6A-02a,probably,RES2047,Roodt,Ruby Johnson,70:30 setting
4,2046,Joshua Keener,15/01/2055,CD Experiment,08/01/2055,Fraction,1.00,DCM/MeOH,Marine,1-1*#3R1-01a,know,RES2046,Roodt,Ruby Johnson,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2041,Ruby Johnson,14/06/2051,CD Experiment,07/06/2051,Fraction from flash,0.50,MeOH,Marine,TIC2041-12RJ-03a,various,RES2041,Roodt,Ruby Johnson,Used compound G in fractionation
76,2042,Ruby Johnson,01/11/2050,RU LC/MS,25/10/2050,SPE fraction,0.25,DCM/MeOH,Marine,MOZ42-9RJ-02a,game,RES2042,Smith,Priscilla Evans,None
77,2047,Ruby Johnson,26/07/2051,RU LC/MS,19/07/2051,Fraction,0.10,MeOH,Microbial,AMRU4A-01d,myself,RES2047,Smith,Ruby Johnson,Used compound G in fractionation
78,2046,Joshua Keener,07/11/2050,RU LC-MS & 1H-NMR,31/10/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,Marine,1-1*#2R1-02a,structure,RES2046,Smith,Ruby Johnson,None


In [377]:
numSampled3 = 20
subsubfraction1_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1231, ignore_index=True)
subsubfraction2_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1232, ignore_index=True)
subsubfraction3_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1233, ignore_index=True)
subsubfraction4_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1234, ignore_index=True)
subsubfraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2045,Ruby Johnson,12/03/2051,RU Bioassay,05/03/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3B-01b,day,RES2045,Roodt,Priscilla Evans,None
1,2042,Joshua Keener,01/04/2055,RU LC-MS & 1H-NMR,25/03/2055,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01a,pay,RES2042,Smith,Priscilla Evans,Used compound F in fractionation
2,2045,Ruby Johnson,12/03/2051,RU LC-MS & 1H-NMR,05/03/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Microbial,AMRU3B-02b,day,RES2045,Smith,Amanda Bowker,None
3,2046,Ruby Johnson,27/10/2052,RU LC-MS & 1H-NMR,20/10/2052,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01c,difficult,RES2046,Roodt,Amanda Bowker,20:80 setting
4,2041,Joshua Keener,19/08/2054,RU Bioassay,12/08/2054,Fraction,20.00,DCM/MeOH,Marine,TIC2041-7RH-01a,stop,RES2041,Roodt,Priscilla Evans,20:80 setting
5,2046,Joshua Keener,15/01/2055,CD Experiment,08/01/2055,Fraction,1.00,DCM/MeOH,Marine,1-1*#3R1-01a,know,RES2046,Roodt,Ruby Johnson,None
6,2047,Ruby Johnson,16/12/2053,CD Experiment,09/12/2053,SPE fraction,20.00,DCM/MeOH,Microbial,AMRU6A-02a,probably,RES2047,Roodt,Ruby Johnson,70:30 setting
7,2042,Ruby Johnson,31/03/2051,Rhodes MS,24/03/2051,SPE fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-1JR-01a,next,RES2042,Smith,Ruby Johnson,None
8,2046,Ruby Johnson,21/05/2050,Rhodes MS,14/05/2050,Fraction,0.25,MeOH,Marine,1-1*#1R2-01a,dark,RES2046,Smith,Ruby Johnson,Used compound F in fractionation
9,2041,Ruby Johnson,14/06/2051,CD Experiment,07/06/2051,Fraction from flash,0.50,MeOH,Marine,TIC2041-12RJ-03a,various,RES2041,Roodt,Ruby Johnson,Used compound G in fractionation


In [378]:
fractionThird_df = pd.concat([subsubfraction1_df, subsubfraction2_df, subsubfraction3_df, subsubfraction4_df], ignore_index=True)
fractionThird_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2045,Ruby Johnson,12/03/2051,RU Bioassay,05/03/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3B-01b,day,RES2045,Roodt,Priscilla Evans,None
1,2042,Joshua Keener,01/04/2055,RU LC-MS & 1H-NMR,25/03/2055,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01a,pay,RES2042,Smith,Priscilla Evans,Used compound F in fractionation
2,2045,Ruby Johnson,12/03/2051,RU LC-MS & 1H-NMR,05/03/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Microbial,AMRU3B-02b,day,RES2045,Smith,Amanda Bowker,None
3,2046,Ruby Johnson,27/10/2052,RU LC-MS & 1H-NMR,20/10/2052,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01c,difficult,RES2046,Roodt,Amanda Bowker,20:80 setting
4,2041,Joshua Keener,19/08/2054,RU Bioassay,12/08/2054,Fraction,20.00,DCM/MeOH,Marine,TIC2041-7RH-01a,stop,RES2041,Roodt,Priscilla Evans,20:80 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2046,Ruby Johnson,30/01/2052,RU LC/MS,23/01/2052,Fraction from flash,20.00,DCM:MeOH 2:1,Marine,1-1*#2R2-01c,last,RES2046,Roodt,Ruby Johnson,None
76,2045,Ruby Johnson,08/04/2051,RU Bioassay,01/04/2051,HPLC Fraction,0.50,DCM/MeOH,Microbial,AMRU7B-01b,company,RES2045,Smith,Ruby Johnson,Used compound G in fractionation
77,2042,Joshua Keener,27/12/2053,CD Experiment,20/12/2053,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01a,what,RES2042,Roodt,Amanda Bowker,20:80 setting
78,2047,Ruby Johnson,26/07/2051,RU LC-MS & 1H-NMR,19/07/2051,Fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU4A-01c,myself,RES2047,Smith,Ruby Johnson,None


In [379]:
# Parts that are different per subsubfraction:
# Amount available
fake = Faker()
Faker.seed(1232)
random.seed(1232)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amountSub_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amountsubSub_df = create_amountSub_data(fake, 80)

fractionThird_df = pd.concat([amountsubSub_df, fractionThird_df], axis=1)
fractionThird_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,57.7,2045,Ruby Johnson,12/03/2051,RU Bioassay,05/03/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3B-01b,day,RES2045,Roodt,Priscilla Evans,None
1,70.1,2042,Joshua Keener,01/04/2055,RU LC-MS & 1H-NMR,25/03/2055,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01a,pay,RES2042,Smith,Priscilla Evans,Used compound F in fractionation
2,90.8,2045,Ruby Johnson,12/03/2051,RU LC-MS & 1H-NMR,05/03/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Microbial,AMRU3B-02b,day,RES2045,Smith,Amanda Bowker,None
3,14.2,2046,Ruby Johnson,27/10/2052,RU LC-MS & 1H-NMR,20/10/2052,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01c,difficult,RES2046,Roodt,Amanda Bowker,20:80 setting
4,3.3,2041,Joshua Keener,19/08/2054,RU Bioassay,12/08/2054,Fraction,20.00,DCM/MeOH,Marine,TIC2041-7RH-01a,stop,RES2041,Roodt,Priscilla Evans,20:80 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,42.5,2046,Ruby Johnson,30/01/2052,RU LC/MS,23/01/2052,Fraction from flash,20.00,DCM:MeOH 2:1,Marine,1-1*#2R2-01c,last,RES2046,Roodt,Ruby Johnson,None
76,8.7,2045,Ruby Johnson,08/04/2051,RU Bioassay,01/04/2051,HPLC Fraction,0.50,DCM/MeOH,Microbial,AMRU7B-01b,company,RES2045,Smith,Ruby Johnson,Used compound G in fractionation
77,60.2,2042,Joshua Keener,27/12/2053,CD Experiment,20/12/2053,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01a,what,RES2042,Roodt,Amanda Bowker,20:80 setting
78,60.8,2047,Ruby Johnson,26/07/2051,RU LC-MS & 1H-NMR,19/07/2051,Fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU4A-01c,myself,RES2047,Smith,Ruby Johnson,None


In [380]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
# 
sourceSubSeries = fractionThird_df['sourceID']
listSubSupplier = []
subsupplier_freq = {}

for index, value in sourceSubSeries.items():
    if (value in subsupplier_freq):
        subsupplier_freq[value] += 1
    else:
        subsupplier_freq[value] = 1
    listSubSupplier.append(value + str(chr(subsupplier_freq[value] + 96)))

subsupplierSeries = pd.Series(listSubSupplier)
#print(supplierSeries)

fractionThird_df.insert(1, 'supplierID', subsupplierSeries)
fractionThird_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,57.7,AMRU3B-01ba,2045,Ruby Johnson,12/03/2051,RU Bioassay,05/03/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3B-01b,day,RES2045,Roodt,Priscilla Evans,None
1,70.1,MOZ42-13RH-01aa,2042,Joshua Keener,01/04/2055,RU LC-MS & 1H-NMR,25/03/2055,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01a,pay,RES2042,Smith,Priscilla Evans,Used compound F in fractionation
2,90.8,AMRU3B-02ba,2045,Ruby Johnson,12/03/2051,RU LC-MS & 1H-NMR,05/03/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Microbial,AMRU3B-02b,day,RES2045,Smith,Amanda Bowker,None
3,14.2,1-1*#1R3-01ca,2046,Ruby Johnson,27/10/2052,RU LC-MS & 1H-NMR,20/10/2052,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01c,difficult,RES2046,Roodt,Amanda Bowker,20:80 setting
4,3.3,TIC2041-7RH-01aa,2041,Joshua Keener,19/08/2054,RU Bioassay,12/08/2054,Fraction,20.00,DCM/MeOH,Marine,TIC2041-7RH-01a,stop,RES2041,Roodt,Priscilla Evans,20:80 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,42.5,1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,RU LC/MS,23/01/2052,Fraction from flash,20.00,DCM:MeOH 2:1,Marine,1-1*#2R2-01c,last,RES2046,Roodt,Ruby Johnson,None
76,8.7,AMRU7B-01ba,2045,Ruby Johnson,08/04/2051,RU Bioassay,01/04/2051,HPLC Fraction,0.50,DCM/MeOH,Microbial,AMRU7B-01b,company,RES2045,Smith,Ruby Johnson,Used compound G in fractionation
77,60.2,MOZ42-15RH-01ad,2042,Joshua Keener,27/12/2053,CD Experiment,20/12/2053,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01a,what,RES2042,Roodt,Amanda Bowker,20:80 setting
78,60.8,AMRU4A-01ca,2047,Ruby Johnson,26/07/2051,RU LC-MS & 1H-NMR,19/07/2051,Fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU4A-01c,myself,RES2047,Smith,Ruby Johnson,None


In [381]:
#Get 20 records that will be pure, rather than just fractions
pureFirst_df = fractionThird_df.sample(frac = 0.25, random_state=1234)
pureFirst_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
46,88.2,MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,CD Experiment,25/03/2055,SPE fraction,0.10,MeOH,Marine,MOZ42-13RH-02b,pay,RES2042,Smith,Ruby Johnson,Used compound G in fractionation
21,29.4,1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,Rhodes MS,23/01/2052,Fraction from flash,1.00,MeOH,Marine,1-1*#2R2-02a,last,RES2046,Smith,Priscilla Evans,70:30 setting
10,13.6,AMRU5B-01ba,2045,Joshua Keener,04/10/2051,RU LC/MS,27/09/2051,SPE fraction,0.10,DCM/MeOH,Microbial,AMRU5B-01b,herself,RES2045,Roodt,Ruby Johnson,None
54,67.3,MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,Rhodes MS,24/03/2051,SPE fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-1JR-01a,next,RES2042,Smith,Ruby Johnson,None
29,39.7,MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,RU LC/MS,12/07/2050,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-3RH-02a,memory,RES2042,Roodt,Ruby Johnson,None
67,75.8,AMRU5B-01ab,2045,Joshua Keener,04/10/2051,CD Experiment,27/09/2051,SPE fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU5B-01a,herself,RES2045,Smith,Amanda Bowker,None
75,42.5,1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,RU LC/MS,23/01/2052,Fraction from flash,20.00,DCM:MeOH 2:1,Marine,1-1*#2R2-01c,last,RES2046,Roodt,Ruby Johnson,None
45,9.6,MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,CD Experiment,24/03/2051,Fraction from flash,50.00,DCM/MeOH,Marine,MOZ42-1JR-01b,next,RES2042,Roodt,Priscilla Evans,Used compound G in fractionation
18,12.9,SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,RU Bioassay,17/11/2050,SPE fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-5RH-01b,attorney,RES2043,Roodt,Priscilla Evans,20:80 setting
68,28.7,SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,CD Experiment,25/08/2054,Fraction,50.00,DCM:MeOH 2:1,Marine,SAF43-12JR-01a,spring,RES2043,Smith,Amanda Bowker,Used compound G in fractionation


In [382]:
fractionThird_df = fractionThird_df.drop(pureFirst_df.index)
fractionThird_df.reset_index(drop=True, inplace=True)
fractionThird_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,57.7,AMRU3B-01ba,2045,Ruby Johnson,12/03/2051,RU Bioassay,05/03/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3B-01b,day,RES2045,Roodt,Priscilla Evans,None
1,70.1,MOZ42-13RH-01aa,2042,Joshua Keener,01/04/2055,RU LC-MS & 1H-NMR,25/03/2055,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01a,pay,RES2042,Smith,Priscilla Evans,Used compound F in fractionation
2,90.8,AMRU3B-02ba,2045,Ruby Johnson,12/03/2051,RU LC-MS & 1H-NMR,05/03/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Microbial,AMRU3B-02b,day,RES2045,Smith,Amanda Bowker,None
3,14.2,1-1*#1R3-01ca,2046,Ruby Johnson,27/10/2052,RU LC-MS & 1H-NMR,20/10/2052,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01c,difficult,RES2046,Roodt,Amanda Bowker,20:80 setting
4,95.3,1-1*#3R1-01aa,2046,Joshua Keener,15/01/2055,CD Experiment,08/01/2055,Fraction,1.00,DCM/MeOH,Marine,1-1*#3R1-01a,know,RES2046,Roodt,Ruby Johnson,None
5,72.9,AMRU6A-02aa,2047,Ruby Johnson,16/12/2053,CD Experiment,09/12/2053,SPE fraction,20.00,DCM/MeOH,Microbial,AMRU6A-02a,probably,RES2047,Roodt,Ruby Johnson,70:30 setting
6,50.7,MOZ42-1JR-01aa,2042,Ruby Johnson,31/03/2051,Rhodes MS,24/03/2051,SPE fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-1JR-01a,next,RES2042,Smith,Ruby Johnson,None
7,10.0,1-1*#1R2-01aa,2046,Ruby Johnson,21/05/2050,Rhodes MS,14/05/2050,Fraction,0.25,MeOH,Marine,1-1*#1R2-01a,dark,RES2046,Smith,Ruby Johnson,Used compound F in fractionation
8,1.3,TIC2041-12RJ-03aa,2041,Ruby Johnson,14/06/2051,CD Experiment,07/06/2051,Fraction from flash,0.50,MeOH,Marine,TIC2041-12RJ-03a,various,RES2041,Roodt,Ruby Johnson,Used compound G in fractionation
9,62.0,TIC2041-5RH-01ba,2041,Joshua Keener,06/07/2050,RU LC/MS,29/06/2050,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-5RH-01b,put,RES2041,Smith,Amanda Bowker,None


In [383]:
offset = len(fractionFirst_df) + len(fractionSecond_df)
fractionThird_df.insert(0, 'mnpID', range(offset + 1, 1 + offset + len(fractionThird_df)))
fractionThird_df['mnpID'] = 'MNP' + fractionThird_df['mnpID'].astype(str).str.zfill(5)
# Include level (3)
fractionThird_df.insert(17, 'level', 3)
fractionThird_df

,mnpID,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00161,57.7,AMRU3B-01ba,2045,Ruby Johnson,12/03/2051,RU Bioassay,05/03/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3B-01b,day,RES2045,Roodt,Priscilla Evans,3,None
1,MNP00162,70.1,MOZ42-13RH-01aa,2042,Joshua Keener,01/04/2055,RU LC-MS & 1H-NMR,25/03/2055,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01a,pay,RES2042,Smith,Priscilla Evans,3,Used compound F in fractionation
2,MNP00163,90.8,AMRU3B-02ba,2045,Ruby Johnson,12/03/2051,RU LC-MS & 1H-NMR,05/03/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Microbial,AMRU3B-02b,day,RES2045,Smith,Amanda Bowker,3,None
3,MNP00164,14.2,1-1*#1R3-01ca,2046,Ruby Johnson,27/10/2052,RU LC-MS & 1H-NMR,20/10/2052,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,1-1*#1R3-01c,difficult,RES2046,Roodt,Amanda Bowker,3,20:80 setting
4,MNP00165,95.3,1-1*#3R1-01aa,2046,Joshua Keener,15/01/2055,CD Experiment,08/01/2055,Fraction,1.00,DCM/MeOH,Marine,1-1*#3R1-01a,know,RES2046,Roodt,Ruby Johnson,3,None
5,MNP00166,72.9,AMRU6A-02aa,2047,Ruby Johnson,16/12/2053,CD Experiment,09/12/2053,SPE fraction,20.00,DCM/MeOH,Microbial,AMRU6A-02a,probably,RES2047,Roodt,Ruby Johnson,3,70:30 setting
6,MNP00167,50.7,MOZ42-1JR-01aa,2042,Ruby Johnson,31/03/2051,Rhodes MS,24/03/2051,SPE fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-1JR-01a,next,RES2042,Smith,Ruby Johnson,3,None
7,MNP00168,10.0,1-1*#1R2-01aa,2046,Ruby Johnson,21/05/2050,Rhodes MS,14/05/2050,Fraction,0.25,MeOH,Marine,1-1*#1R2-01a,dark,RES2046,Smith,Ruby Johnson,3,Used compound F in fractionation
8,MNP00169,1.3,TIC2041-12RJ-03aa,2041,Ruby Johnson,14/06/2051,CD Experiment,07/06/2051,Fraction from flash,0.50,MeOH,Marine,TIC2041-12RJ-03a,various,RES2041,Roodt,Ruby Johnson,3,Used compound G in fractionation
9,MNP00170,62.0,TIC2041-5RH-01ba,2041,Joshua Keener,06/07/2050,RU LC/MS,29/06/2050,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-5RH-01b,put,RES2041,Smith,Amanda Bowker,3,None


Subsubsubfraction

In [384]:
sub3fraction_df = fractionThird_df.copy(deep=True)
sub3fraction_df = sub3fraction_df[["supplierID", "year", "assignedDate", "species", "permitNo", "project"]]
sub3fraction_df

,supplierID,year,assignedDate,species,permitNo,project
0,AMRU3B-01ba,2045,12/03/2051,day,RES2045,Microbial
1,MOZ42-13RH-01aa,2042,01/04/2055,pay,RES2042,Marine
2,AMRU3B-02ba,2045,12/03/2051,day,RES2045,Microbial
3,1-1*#1R3-01ca,2046,27/10/2052,difficult,RES2046,Marine
4,1-1*#3R1-01aa,2046,15/01/2055,know,RES2046,Marine
5,AMRU6A-02aa,2047,16/12/2053,probably,RES2047,Microbial
6,MOZ42-1JR-01aa,2042,31/03/2051,next,RES2042,Marine
7,1-1*#1R2-01aa,2046,21/05/2050,dark,RES2046,Marine
8,TIC2041-12RJ-03aa,2041,14/06/2051,various,RES2041,Marine
9,TIC2041-5RH-01ba,2041,06/07/2050,put,RES2041,Marine


In [385]:
# Inserting assignedDate and submittedDate
sub3fraction_df.assignedDate = pd.to_datetime(sub3fraction_df.assignedDate, format='%d/%m/%Y')
sub3fraction_df.insert(3, 'submittedDate', sub3fraction_df['assignedDate'])
# Date submitted - do 3 weeks after previous assignedDate
sub3fraction_df['submittedDate'] = sub3fraction_df['submittedDate'] + timedelta(days=21)
sub3fraction_df['submittedDate'] = sub3fraction_df['submittedDate'].dt.strftime('%d/%m/%Y')
# Date assigned - do a week after submittedDate
sub3fraction_df['assignedDate'] = sub3fraction_df['assignedDate'] + timedelta(days=28)
sub3fraction_df['assignedDate'] = sub3fraction_df['assignedDate'].dt.strftime('%d/%m/%Y')
sub3fraction_df

,supplierID,year,assignedDate,submittedDate,species,permitNo,project
0,AMRU3B-01ba,2045,09/04/2051,02/04/2051,day,RES2045,Microbial
1,MOZ42-13RH-01aa,2042,29/04/2055,22/04/2055,pay,RES2042,Marine
2,AMRU3B-02ba,2045,09/04/2051,02/04/2051,day,RES2045,Microbial
3,1-1*#1R3-01ca,2046,24/11/2052,17/11/2052,difficult,RES2046,Marine
4,1-1*#3R1-01aa,2046,12/02/2055,05/02/2055,know,RES2046,Marine
5,AMRU6A-02aa,2047,13/01/2054,06/01/2054,probably,RES2047,Microbial
6,MOZ42-1JR-01aa,2042,28/04/2051,21/04/2051,next,RES2042,Marine
7,1-1*#1R2-01aa,2046,18/06/2050,11/06/2050,dark,RES2046,Marine
8,TIC2041-12RJ-03aa,2041,12/07/2051,05/07/2051,various,RES2041,Marine
9,TIC2041-5RH-01ba,2041,03/08/2050,27/07/2050,put,RES2041,Marine


In [386]:
fake = Faker()
Faker.seed(1232)
random.seed(1232)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound Q in fractionation', 'Used compound R in fractionation', None, None, '10:90 setting', 'Combination setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_sub3fraction_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

sub3fractionSubset_df = create_sub3fraction_data(fake, len(sub3fraction_df))
sub3fractionSubset_df

,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Ruby Johnson,RU Bioassay,HPLC Fraction,Roodt,Priscilla Evans,MeOH,20.00,Used compound Q in fractionation
1,Joshua Keener,CD Experiment,Fraction,Roodt,Amanda Bowker,DCM/MeOH,50.00,None
2,Joshua Keener,Rhodes MS,SPE fraction,Smith,Priscilla Evans,MeOH,1.00,None
3,Ruby Johnson,RU LC-MS & 1H-NMR,Fraction from flash,Smith,Ruby Johnson,MeOH,20.00,None
4,Ruby Johnson,RU Bioassay,Fraction from flash,Smith,Priscilla Evans,DCM:MeOH 2:1,0.25,None
5,Joshua Keener,RU Bioassay,HPLC Fraction,Smith,Amanda Bowker,DCM:MeOH 2:1,0.50,Combination setting
6,Ruby Johnson,CD Experiment,Fraction from flash,Roodt,Ruby Johnson,DCM:MeOH 2:1,0.25,None
7,Ruby Johnson,Rhodes MS,SPE fraction,Roodt,Amanda Bowker,DCM/MeOH,0.50,None
8,Ruby Johnson,RU LC/MS,Fraction from flash,Smith,Priscilla Evans,DCM:MeOH 2:1,0.10,None
9,Ruby Johnson,RU LC/MS,Fraction,Roodt,Ruby Johnson,DCM/MeOH,20.00,None


In [387]:
newSub3Fraction_df = pd.concat([sub3fraction_df, sub3fractionSubset_df], axis=1)
newSub3Fraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'project', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newSub3Fraction_df = newSub3Fraction_df[orderCols]
newSub3Fraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2045,Ruby Johnson,09/04/2051,RU Bioassay,02/04/2051,HPLC Fraction,20.00,MeOH,Microbial,AMRU3B-01ba,day,RES2045,Roodt,Priscilla Evans,Used compound Q in fractionation
1,2042,Joshua Keener,29/04/2055,CD Experiment,22/04/2055,Fraction,50.00,DCM/MeOH,Marine,MOZ42-13RH-01aa,pay,RES2042,Roodt,Amanda Bowker,None
2,2045,Joshua Keener,09/04/2051,Rhodes MS,02/04/2051,SPE fraction,1.00,MeOH,Microbial,AMRU3B-02ba,day,RES2045,Smith,Priscilla Evans,None
3,2046,Ruby Johnson,24/11/2052,RU LC-MS & 1H-NMR,17/11/2052,Fraction from flash,20.00,MeOH,Marine,1-1*#1R3-01ca,difficult,RES2046,Smith,Ruby Johnson,None
4,2046,Ruby Johnson,12/02/2055,RU Bioassay,05/02/2055,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,1-1*#3R1-01aa,know,RES2046,Smith,Priscilla Evans,None
5,2047,Joshua Keener,13/01/2054,RU Bioassay,06/01/2054,HPLC Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU6A-02aa,probably,RES2047,Smith,Amanda Bowker,Combination setting
6,2042,Ruby Johnson,28/04/2051,CD Experiment,21/04/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,MOZ42-1JR-01aa,next,RES2042,Roodt,Ruby Johnson,None
7,2046,Ruby Johnson,18/06/2050,Rhodes MS,11/06/2050,SPE fraction,0.50,DCM/MeOH,Marine,1-1*#1R2-01aa,dark,RES2046,Roodt,Amanda Bowker,None
8,2041,Ruby Johnson,12/07/2051,RU LC/MS,05/07/2051,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-12RJ-03aa,various,RES2041,Smith,Priscilla Evans,None
9,2041,Ruby Johnson,03/08/2050,RU LC/MS,27/07/2050,Fraction,20.00,DCM/MeOH,Marine,TIC2041-5RH-01ba,put,RES2041,Roodt,Ruby Johnson,None


In [388]:
numSampled4 = 20
sub3fraction1_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1231, ignore_index=True)
sub3fraction2_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1232, ignore_index=True)
sub3fraction3_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1233, ignore_index=True)
sub3fraction4_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1234, ignore_index=True)
sub3fraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2046,Ruby Johnson,18/06/2050,Rhodes MS,11/06/2050,SPE fraction,0.50,DCM/MeOH,Marine,1-1*#1R2-01aa,dark,RES2046,Roodt,Amanda Bowker,None
1,2041,Ruby Johnson,03/08/2050,RU Bioassay,27/07/2050,Fraction from flash,0.25,DCM/MeOH,Marine,TIC2041-5RH-01bc,put,RES2041,Smith,Priscilla Evans,None
2,2046,Ruby Johnson,24/11/2052,RU LC-MS & 1H-NMR,17/11/2052,Fraction from flash,20.00,MeOH,Marine,1-1*#1R3-01ca,difficult,RES2046,Smith,Ruby Johnson,None
3,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
4,2041,Ruby Johnson,16/09/2054,CD Experiment,09/09/2054,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-7RH-01ab,stop,RES2041,Roodt,Amanda Bowker,None
5,2046,Ruby Johnson,12/02/2055,RU Bioassay,05/02/2055,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,1-1*#3R1-01aa,know,RES2046,Smith,Priscilla Evans,None
6,2045,Ruby Johnson,09/04/2051,RU LC-MS & 1H-NMR,02/04/2051,Fraction,0.50,MeOH,Microbial,AMRU3B-02ab,day,RES2045,Smith,Ruby Johnson,None
7,2045,Ruby Johnson,09/04/2051,RU Bioassay,02/04/2051,HPLC Fraction,20.00,MeOH,Microbial,AMRU3B-01ba,day,RES2045,Roodt,Priscilla Evans,Used compound Q in fractionation
8,2042,Joshua Keener,04/08/2051,RU Bioassay,28/07/2051,Fraction,50.00,MeOH,Marine,MOZ42-5JR-01ba,scene,RES2042,Roodt,Priscilla Evans,None
9,2046,Ruby Johnson,05/12/2050,RU Bioassay,28/11/2050,HPLC Fraction,1.00,DCM/MeOH,Marine,1-1*#2R1-01aa,structure,RES2046,Smith,Amanda Bowker,None


In [389]:
fractionFourth_df = pd.concat([sub3fraction1_df, sub3fraction2_df, sub3fraction3_df, sub3fraction4_df], ignore_index=True)
fractionFourth_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2046,Ruby Johnson,18/06/2050,Rhodes MS,11/06/2050,SPE fraction,0.50,DCM/MeOH,Marine,1-1*#1R2-01aa,dark,RES2046,Roodt,Amanda Bowker,None
1,2041,Ruby Johnson,03/08/2050,RU Bioassay,27/07/2050,Fraction from flash,0.25,DCM/MeOH,Marine,TIC2041-5RH-01bc,put,RES2041,Smith,Priscilla Evans,None
2,2046,Ruby Johnson,24/11/2052,RU LC-MS & 1H-NMR,17/11/2052,Fraction from flash,20.00,MeOH,Marine,1-1*#1R3-01ca,difficult,RES2046,Smith,Ruby Johnson,None
3,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
4,2041,Ruby Johnson,16/09/2054,CD Experiment,09/09/2054,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-7RH-01ab,stop,RES2041,Roodt,Amanda Bowker,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2045,Joshua Keener,22/01/2055,CD Experiment,15/01/2055,HPLC Fraction,0.10,DCM/MeOH,Microbial,AMRU1C-01aa,bar,RES2045,Roodt,Priscilla Evans,None
76,2045,Joshua Keener,06/05/2051,RU Bioassay,29/04/2051,HPLC Fraction,50.00,DCM/MeOH,Microbial,AMRU7B-02aa,company,RES2045,Roodt,Ruby Johnson,Used compound Q in fractionation
77,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
78,2042,Joshua Keener,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01aa,what,RES2042,Smith,Ruby Johnson,10:90 setting


In [390]:
# Parts that are different per subsubsubfraction:
# Amount available
fake = Faker()
Faker.seed(1231)
random.seed(1231)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amountSub_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amount3Sub_df = create_amountSub_data(fake, 80)

fractionFourth_df = pd.concat([amount3Sub_df, fractionFourth_df], axis=1)
fractionFourth_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,37.9,2046,Ruby Johnson,18/06/2050,Rhodes MS,11/06/2050,SPE fraction,0.50,DCM/MeOH,Marine,1-1*#1R2-01aa,dark,RES2046,Roodt,Amanda Bowker,None
1,25.7,2041,Ruby Johnson,03/08/2050,RU Bioassay,27/07/2050,Fraction from flash,0.25,DCM/MeOH,Marine,TIC2041-5RH-01bc,put,RES2041,Smith,Priscilla Evans,None
2,41.6,2046,Ruby Johnson,24/11/2052,RU LC-MS & 1H-NMR,17/11/2052,Fraction from flash,20.00,MeOH,Marine,1-1*#1R3-01ca,difficult,RES2046,Smith,Ruby Johnson,None
3,39.4,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
4,51.5,2041,Ruby Johnson,16/09/2054,CD Experiment,09/09/2054,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-7RH-01ab,stop,RES2041,Roodt,Amanda Bowker,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,52.7,2045,Joshua Keener,22/01/2055,CD Experiment,15/01/2055,HPLC Fraction,0.10,DCM/MeOH,Microbial,AMRU1C-01aa,bar,RES2045,Roodt,Priscilla Evans,None
76,70.0,2045,Joshua Keener,06/05/2051,RU Bioassay,29/04/2051,HPLC Fraction,50.00,DCM/MeOH,Microbial,AMRU7B-02aa,company,RES2045,Roodt,Ruby Johnson,Used compound Q in fractionation
77,19.3,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
78,33.7,2042,Joshua Keener,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01aa,what,RES2042,Smith,Ruby Johnson,10:90 setting


In [391]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
# 
sourceSubSeries = fractionFourth_df['sourceID']
listSubSupplier = []
subsupplier_freq = {}

for index, value in sourceSubSeries.items():
    if (value in subsupplier_freq):
        subsupplier_freq[value] += 1
    else:
        subsupplier_freq[value] = 1
    listSubSupplier.append(value + str(chr(subsupplier_freq[value] + 96)))

subsupplierSeries = pd.Series(listSubSupplier)
#print(supplierSeries)

fractionFourth_df.insert(1, 'supplierID', subsupplierSeries)
fractionFourth_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,37.9,1-1*#1R2-01aaa,2046,Ruby Johnson,18/06/2050,Rhodes MS,11/06/2050,SPE fraction,0.50,DCM/MeOH,Marine,1-1*#1R2-01aa,dark,RES2046,Roodt,Amanda Bowker,None
1,25.7,TIC2041-5RH-01bca,2041,Ruby Johnson,03/08/2050,RU Bioassay,27/07/2050,Fraction from flash,0.25,DCM/MeOH,Marine,TIC2041-5RH-01bc,put,RES2041,Smith,Priscilla Evans,None
2,41.6,1-1*#1R3-01caa,2046,Ruby Johnson,24/11/2052,RU LC-MS & 1H-NMR,17/11/2052,Fraction from flash,20.00,MeOH,Marine,1-1*#1R3-01ca,difficult,RES2046,Smith,Ruby Johnson,None
3,39.4,MOZ42-1JR-01bba,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
4,51.5,TIC2041-7RH-01aba,2041,Ruby Johnson,16/09/2054,CD Experiment,09/09/2054,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-7RH-01ab,stop,RES2041,Roodt,Amanda Bowker,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,52.7,AMRU1C-01aac,2045,Joshua Keener,22/01/2055,CD Experiment,15/01/2055,HPLC Fraction,0.10,DCM/MeOH,Microbial,AMRU1C-01aa,bar,RES2045,Roodt,Priscilla Evans,None
76,70.0,AMRU7B-02aab,2045,Joshua Keener,06/05/2051,RU Bioassay,29/04/2051,HPLC Fraction,50.00,DCM/MeOH,Microbial,AMRU7B-02aa,company,RES2045,Roodt,Ruby Johnson,Used compound Q in fractionation
77,19.3,MOZ42-1JR-01bbc,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
78,33.7,MOZ42-15RH-01aac,2042,Joshua Keener,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01aa,what,RES2042,Smith,Ruby Johnson,10:90 setting


In [392]:
#Get 40 records that will be pure, rather than just fractions
pureSecond_df = fractionFourth_df.sample(frac = 0.5, random_state=1233)
pureSecond_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
27,84.0,AMRU3B-01cbb,2045,Joshua Keener,09/04/2051,RU Bioassay,02/04/2051,Fraction,0.10,DCM/MeOH,Microbial,AMRU3B-01cb,day,RES2045,Roodt,Priscilla Evans,10:90 setting
10,84.5,AMRU3B-01cba,2045,Joshua Keener,09/04/2051,RU Bioassay,02/04/2051,Fraction,0.10,DCM/MeOH,Microbial,AMRU3B-01cb,day,RES2045,Roodt,Priscilla Evans,10:90 setting
21,83.2,MOZ42-1JR-01bbb,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
25,56.5,SAF43-4RH-01baa,2043,Ruby Johnson,08/05/2054,CD Experiment,01/05/2054,SPE fraction,0.50,MeOH,Marine,SAF43-4RH-01ba,free,RES2043,Smith,Amanda Bowker,None
3,39.4,MOZ42-1JR-01bba,2042,Ruby Johnson,28/04/2051,Rhodes MS,21/04/2051,HPLC Fraction,20.00,DCM/MeOH,Marine,MOZ42-1JR-01bb,next,RES2042,Smith,Amanda Bowker,Used compound R in fractionation
54,2.1,TIC2041-8RJ-03aaa,2041,Joshua Keener,08/03/2054,RU LC-MS & 1H-NMR,01/03/2054,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,TIC2041-8RJ-03aa,discuss,RES2041,Smith,Priscilla Evans,Combination setting
55,50.9,MOZ42-15RH-01abc,2042,Ruby Johnson,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction,50.00,DCM/MeOH,Marine,MOZ42-15RH-01ab,what,RES2042,Roodt,Ruby Johnson,Used compound R in fractionation
53,28.9,AMRU4A-01cab,2047,Ruby Johnson,23/08/2051,RU Bioassay,16/08/2051,Fraction,0.10,DCM/MeOH,Microbial,AMRU4A-01ca,myself,RES2047,Roodt,Ruby Johnson,None
44,70.5,TIC2041-12RJ-03aab,2041,Ruby Johnson,12/07/2051,RU LC/MS,05/07/2051,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-12RJ-03aa,various,RES2041,Smith,Priscilla Evans,None
48,31.9,MOZ42-15RH-01aab,2042,Joshua Keener,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-15RH-01aa,what,RES2042,Smith,Ruby Johnson,10:90 setting


In [393]:
fractionFourth_df = fractionFourth_df.drop(pureSecond_df.index)
fractionFourth_df.reset_index(drop=True, inplace=True)
fractionFourth_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,25.7,TIC2041-5RH-01bca,2041,Ruby Johnson,03/08/2050,RU Bioassay,27/07/2050,Fraction from flash,0.25,DCM/MeOH,Marine,TIC2041-5RH-01bc,put,RES2041,Smith,Priscilla Evans,None
1,51.5,TIC2041-7RH-01aba,2041,Ruby Johnson,16/09/2054,CD Experiment,09/09/2054,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-7RH-01ab,stop,RES2041,Roodt,Amanda Bowker,None
2,18.2,1-1*#3R1-01aaa,2046,Ruby Johnson,12/02/2055,RU Bioassay,05/02/2055,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,1-1*#3R1-01aa,know,RES2046,Smith,Priscilla Evans,None
3,25.8,AMRU3B-02aba,2045,Ruby Johnson,09/04/2051,RU LC-MS & 1H-NMR,02/04/2051,Fraction,0.50,MeOH,Microbial,AMRU3B-02ab,day,RES2045,Smith,Ruby Johnson,None
4,74.6,AMRU1C-01aaa,2045,Joshua Keener,22/01/2055,CD Experiment,15/01/2055,HPLC Fraction,0.10,DCM/MeOH,Microbial,AMRU1C-01aa,bar,RES2045,Roodt,Priscilla Evans,None
5,57.4,MOZ42-13RH-02aaa,2042,Ruby Johnson,29/04/2055,RU Bioassay,22/04/2055,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-13RH-02aa,pay,RES2042,Roodt,Ruby Johnson,Combination setting
6,15.3,TIC2041-5RH-02caa,2041,Joshua Keener,03/08/2050,RU LC/MS,27/07/2050,SPE fraction,0.50,MeOH,Marine,TIC2041-5RH-02ca,put,RES2041,Roodt,Ruby Johnson,None
7,65.7,MOZ42-15RH-01aba,2042,Ruby Johnson,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction,50.00,DCM/MeOH,Marine,MOZ42-15RH-01ab,what,RES2042,Roodt,Ruby Johnson,Used compound R in fractionation
8,19.7,MOZ42-1JR-01aaa,2042,Ruby Johnson,28/04/2051,CD Experiment,21/04/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,MOZ42-1JR-01aa,next,RES2042,Roodt,Ruby Johnson,None
9,89.3,TIC2041-12RJ-03aaa,2041,Ruby Johnson,12/07/2051,RU LC/MS,05/07/2051,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-12RJ-03aa,various,RES2041,Smith,Priscilla Evans,None


In [394]:
offset = len(fractionFirst_df) + len(fractionSecond_df) + len(fractionThird_df)
fractionFourth_df.insert(0, 'mnpID', range(offset + 1, 1 + offset + len(fractionFourth_df)))
fractionFourth_df['mnpID'] = 'MNP' + fractionFourth_df['mnpID'].astype(str).str.zfill(5)
# Include level (4)
fractionFourth_df.insert(17, 'level', 4)
fractionFourth_df

,mnpID,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00221,25.7,TIC2041-5RH-01bca,2041,Ruby Johnson,03/08/2050,RU Bioassay,27/07/2050,Fraction from flash,0.25,DCM/MeOH,Marine,TIC2041-5RH-01bc,put,RES2041,Smith,Priscilla Evans,4,None
1,MNP00222,51.5,TIC2041-7RH-01aba,2041,Ruby Johnson,16/09/2054,CD Experiment,09/09/2054,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-7RH-01ab,stop,RES2041,Roodt,Amanda Bowker,4,None
2,MNP00223,18.2,1-1*#3R1-01aaa,2046,Ruby Johnson,12/02/2055,RU Bioassay,05/02/2055,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,1-1*#3R1-01aa,know,RES2046,Smith,Priscilla Evans,4,None
3,MNP00224,25.8,AMRU3B-02aba,2045,Ruby Johnson,09/04/2051,RU LC-MS & 1H-NMR,02/04/2051,Fraction,0.50,MeOH,Microbial,AMRU3B-02ab,day,RES2045,Smith,Ruby Johnson,4,None
4,MNP00225,74.6,AMRU1C-01aaa,2045,Joshua Keener,22/01/2055,CD Experiment,15/01/2055,HPLC Fraction,0.10,DCM/MeOH,Microbial,AMRU1C-01aa,bar,RES2045,Roodt,Priscilla Evans,4,None
5,MNP00226,57.4,MOZ42-13RH-02aaa,2042,Ruby Johnson,29/04/2055,RU Bioassay,22/04/2055,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-13RH-02aa,pay,RES2042,Roodt,Ruby Johnson,4,Combination setting
6,MNP00227,15.3,TIC2041-5RH-02caa,2041,Joshua Keener,03/08/2050,RU LC/MS,27/07/2050,SPE fraction,0.50,MeOH,Marine,TIC2041-5RH-02ca,put,RES2041,Roodt,Ruby Johnson,4,None
7,MNP00228,65.7,MOZ42-15RH-01aba,2042,Ruby Johnson,24/01/2054,RU LC-MS & 1H-NMR,17/01/2054,Fraction,50.00,DCM/MeOH,Marine,MOZ42-15RH-01ab,what,RES2042,Roodt,Ruby Johnson,4,Used compound R in fractionation
8,MNP00229,19.7,MOZ42-1JR-01aaa,2042,Ruby Johnson,28/04/2051,CD Experiment,21/04/2051,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,MOZ42-1JR-01aa,next,RES2042,Roodt,Ruby Johnson,4,None
9,MNP00230,89.3,TIC2041-12RJ-03aaa,2041,Ruby Johnson,12/07/2051,RU LC/MS,05/07/2051,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-12RJ-03aa,various,RES2041,Smith,Priscilla Evans,4,None


#### Pure compounds

In [395]:
# Get all pure fractions together
pureComp_df = pd.concat([pureFirst_df, pureSecond_df], ignore_index=True)
pureComp_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,88.2,MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,CD Experiment,25/03/2055,SPE fraction,0.10,MeOH,Marine,MOZ42-13RH-02b,pay,RES2042,Smith,Ruby Johnson,Used compound G in fractionation
1,29.4,1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,Rhodes MS,23/01/2052,Fraction from flash,1.00,MeOH,Marine,1-1*#2R2-02a,last,RES2046,Smith,Priscilla Evans,70:30 setting
2,13.6,AMRU5B-01ba,2045,Joshua Keener,04/10/2051,RU LC/MS,27/09/2051,SPE fraction,0.10,DCM/MeOH,Microbial,AMRU5B-01b,herself,RES2045,Roodt,Ruby Johnson,None
3,67.3,MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,Rhodes MS,24/03/2051,SPE fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-1JR-01a,next,RES2042,Smith,Ruby Johnson,None
4,39.7,MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,RU LC/MS,12/07/2050,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,MOZ42-3RH-02a,memory,RES2042,Roodt,Ruby Johnson,None
5,75.8,AMRU5B-01ab,2045,Joshua Keener,04/10/2051,CD Experiment,27/09/2051,SPE fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU5B-01a,herself,RES2045,Smith,Amanda Bowker,None
6,42.5,1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,RU LC/MS,23/01/2052,Fraction from flash,20.00,DCM:MeOH 2:1,Marine,1-1*#2R2-01c,last,RES2046,Roodt,Ruby Johnson,None
7,9.6,MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,CD Experiment,24/03/2051,Fraction from flash,50.00,DCM/MeOH,Marine,MOZ42-1JR-01b,next,RES2042,Roodt,Priscilla Evans,Used compound G in fractionation
8,12.9,SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,RU Bioassay,17/11/2050,SPE fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-5RH-01b,attorney,RES2043,Roodt,Priscilla Evans,20:80 setting
9,28.7,SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,CD Experiment,25/08/2054,Fraction,50.00,DCM:MeOH 2:1,Marine,SAF43-12JR-01a,spring,RES2043,Smith,Amanda Bowker,Used compound G in fractionation


In [396]:
# Keep from fractions:
# Assignedby, dateAssigned, dateSubmitted, amountAvailable, sourceID, year, species, permit, submittedby, solvent, sampleType

pureComp_df = pureComp_df[["supplierID", "year", "assignedBy", "assignedDate", "submittedDate", "amountAvailable", "sourceID", "species", "permitNo", "submittedBy", "solvent", "sampleType", "comments"]]
pureComp_df

,supplierID,year,assignedBy,assignedDate,submittedDate,amountAvailable,sourceID,species,permitNo,submittedBy,solvent,sampleType,comments
0,MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,25/03/2055,88.2,MOZ42-13RH-02b,pay,RES2042,Ruby Johnson,MeOH,SPE fraction,Used compound G in fractionation
1,1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,23/01/2052,29.4,1-1*#2R2-02a,last,RES2046,Priscilla Evans,MeOH,Fraction from flash,70:30 setting
2,AMRU5B-01ba,2045,Joshua Keener,04/10/2051,27/09/2051,13.6,AMRU5B-01b,herself,RES2045,Ruby Johnson,DCM/MeOH,SPE fraction,None
3,MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,24/03/2051,67.3,MOZ42-1JR-01a,next,RES2042,Ruby Johnson,DCM:MeOH 2:1,SPE fraction,None
4,MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,12/07/2050,39.7,MOZ42-3RH-02a,memory,RES2042,Ruby Johnson,DCM:MeOH 2:1,Fraction from flash,None
5,AMRU5B-01ab,2045,Joshua Keener,04/10/2051,27/09/2051,75.8,AMRU5B-01a,herself,RES2045,Amanda Bowker,DCM:MeOH 2:1,SPE fraction,None
6,1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,23/01/2052,42.5,1-1*#2R2-01c,last,RES2046,Ruby Johnson,DCM:MeOH 2:1,Fraction from flash,None
7,MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,24/03/2051,9.6,MOZ42-1JR-01b,next,RES2042,Priscilla Evans,DCM/MeOH,Fraction from flash,Used compound G in fractionation
8,SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,17/11/2050,12.9,SAF43-5RH-01b,attorney,RES2043,Priscilla Evans,DCM:MeOH 2:1,SPE fraction,20:80 setting
9,SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,25/08/2054,28.7,SAF43-12JR-01a,spring,RES2043,Amanda Bowker,DCM:MeOH 2:1,Fraction,Used compound G in fractionation


In [397]:
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    destinations = ['Rhodes Bioassay', 'Rhodes MS', 'Rhodes Bioassay/Rhodes MS', 'Rhodes Bioassay, PK assay', 'Rhodes MS/NMR', 'Membrane Disruption assay', 'Rhodes LC/MS + NMR & Bioassay']
    binaryChoice = ['Y', 'N']
    researchgroups = ['Smith', 'Howard', 'Ackerman', 'Leer']
    solvents = ['DMSO', 'aq.MeOH, MeOH', 'DCM: MeOH 2:1']
    stereos = ['InterestPC1', 'InterestPC2', 'InterestPC3', 'InterestPC4', 'InterestPC5', 
               'InterestPC6', 'InterestPC7', 'InterestPC8', 'InterestPC9', 'InterestPC10','N/A']
    existingLit = ['Y', 'https://doi.org/comp1', 'https://doi.org/comp2', 'https://doi.org/comp3', 'https://doi.org/comp4', 'https://doi.org/comp5', 'https://doi.org/comp6', 'https://doi.org/comp7']

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def synthesisPotential(self):
        return random.choice(self.binaryChoice)

    def pure(self):
        return "Y"

    def ms(self):
        return random.choice(self.binaryChoice)
    
    def nmr(self):
        return random.choice(self.binaryChoice)
    
    def sourceType(self):
        return "MNP"

    def researchGroup(self):
        return random.choice(self.researchgroups)

    def solubility(self):
        return random.choice(self.solvents)

    def stereoComments(self):
        return random.choice(self.stereos)

    def smileStructure(self):
        return fake.word()

    def mw(self):
        return (round (random.uniform(0.0, 1.0)*400, 1))

    def additionalMetadata(self):
        return random.choice(self.binaryChoice)

    def existingPatent(self):
        return "Y"

    def existingLiterature(self):
        return random.choice(self.existingLit)

# Add the Provider to our faker object
fake.add_provider(Provider)

# Change:
# AMR ID, supplierID, destination, synthesis, sourceType, pure, ms, nmr, researchGroup, solubility, 
# stereoComments, SMILE, mw, additionalMetadata, existingPatent, existingLiterature

def create_pure_data(fake, no_of_rows):
    columns = ['destinationOfSample', 'synthesisPotential', 'pure', 'ms', 'nmr', 'sourceType',
                'researchGroup', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

pureRest_df = create_pure_data(fake, len(pureComp_df))
pureRest_df

,destinationOfSample,synthesisPotential,pure,ms,nmr,sourceType,researchGroup,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature
0,Rhodes Bioassay/Rhodes MS,N,Y,Y,N,MNP,Howard,DMSO,InterestPC5,former,126.2,N,Y,https://doi.org/comp3
1,Rhodes Bioassay/Rhodes MS,Y,Y,Y,Y,MNP,Smith,"aq.MeOH, MeOH",InterestPC10,fish,280.6,Y,Y,Y
2,Rhodes MS/NMR,Y,Y,N,Y,MNP,Leer,DCM: MeOH 2:1,InterestPC1,Mr,115.1,Y,Y,https://doi.org/comp4
3,Rhodes Bioassay/Rhodes MS,N,Y,N,Y,MNP,Ackerman,DMSO,InterestPC8,five,26.9,N,Y,https://doi.org/comp3
4,Rhodes MS,N,Y,N,N,MNP,Leer,"aq.MeOH, MeOH",InterestPC10,crime,215.5,N,Y,https://doi.org/comp6
5,Membrane Disruption assay,Y,Y,N,N,MNP,Smith,DMSO,InterestPC9,reach,214.9,N,Y,https://doi.org/comp7
6,Rhodes Bioassay/Rhodes MS,N,Y,Y,Y,MNP,Smith,DCM: MeOH 2:1,InterestPC8,father,206.6,N,Y,https://doi.org/comp3
7,Membrane Disruption assay,N,Y,N,Y,MNP,Ackerman,"aq.MeOH, MeOH",N/A,really,214.2,Y,Y,https://doi.org/comp4
8,Membrane Disruption assay,N,Y,Y,Y,MNP,Leer,"aq.MeOH, MeOH",InterestPC4,religious,204.7,N,Y,Y
9,Rhodes Bioassay,Y,Y,Y,Y,MNP,Howard,DCM: MeOH 2:1,InterestPC2,agent,20.1,N,Y,https://doi.org/comp6


In [398]:
# Concatenating the pure compound columns
pure_df = pd.concat([pureComp_df, pureRest_df], axis=1)
orderPure = ['supplierID', 'year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'amountAvailable', 'synthesisPotential', 'sourceID', 'sourceType', 'species', 'permitNo', 'pure', 'ms', 'nmr', 'researchGroup', 'submittedBy', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature', 'solvent', 'sampleType', 'comments']
pure_df = pure_df[orderPure]
pure_df

,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,sourceType,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,Rhodes Bioassay/Rhodes MS,25/03/2055,88.2,N,MOZ42-13RH-02b,MNP,...,DMSO,InterestPC5,former,126.2,N,Y,https://doi.org/comp3,MeOH,SPE fraction,Used compound G in fractionation
1,1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,29.4,Y,1-1*#2R2-02a,MNP,...,"aq.MeOH, MeOH",InterestPC10,fish,280.6,Y,Y,Y,MeOH,Fraction from flash,70:30 setting
2,AMRU5B-01ba,2045,Joshua Keener,04/10/2051,Rhodes MS/NMR,27/09/2051,13.6,Y,AMRU5B-01b,MNP,...,DCM: MeOH 2:1,InterestPC1,Mr,115.1,Y,Y,https://doi.org/comp4,DCM/MeOH,SPE fraction,None
3,MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,Rhodes Bioassay/Rhodes MS,24/03/2051,67.3,N,MOZ42-1JR-01a,MNP,...,DMSO,InterestPC8,five,26.9,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,SPE fraction,None
4,MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,Rhodes MS,12/07/2050,39.7,N,MOZ42-3RH-02a,MNP,...,"aq.MeOH, MeOH",InterestPC10,crime,215.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction from flash,None
5,AMRU5B-01ab,2045,Joshua Keener,04/10/2051,Membrane Disruption assay,27/09/2051,75.8,Y,AMRU5B-01a,MNP,...,DMSO,InterestPC9,reach,214.9,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,SPE fraction,None
6,1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,42.5,N,1-1*#2R2-01c,MNP,...,DCM: MeOH 2:1,InterestPC8,father,206.6,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,Fraction from flash,None
7,MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,Membrane Disruption assay,24/03/2051,9.6,N,MOZ42-1JR-01b,MNP,...,"aq.MeOH, MeOH",N/A,really,214.2,Y,Y,https://doi.org/comp4,DCM/MeOH,Fraction from flash,Used compound G in fractionation
8,SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,Membrane Disruption assay,17/11/2050,12.9,N,SAF43-5RH-01b,MNP,...,"aq.MeOH, MeOH",InterestPC4,religious,204.7,N,Y,Y,DCM:MeOH 2:1,SPE fraction,20:80 setting
9,SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,Rhodes Bioassay,25/08/2054,28.7,Y,SAF43-12JR-01a,MNP,...,DCM: MeOH 2:1,InterestPC2,agent,20.1,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,Used compound G in fractionation


In [399]:
# Still to do: AMR ID, supplierID, existingPatent, existingLiterature
# Make AMR IDs
pureOffset = len(original_df)
pure_df.insert(0, 'amrID', range(pureOffset + 1, 1 + pureOffset + len(pure_df)))
pure_df['amrID'] = 'AMR' + pure_df['amrID'].astype(str).str.zfill(5)
pure_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR00101,MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,Rhodes Bioassay/Rhodes MS,25/03/2055,88.2,N,MOZ42-13RH-02b,...,DMSO,InterestPC5,former,126.2,N,Y,https://doi.org/comp3,MeOH,SPE fraction,Used compound G in fractionation
1,AMR00102,1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,29.4,Y,1-1*#2R2-02a,...,"aq.MeOH, MeOH",InterestPC10,fish,280.6,Y,Y,Y,MeOH,Fraction from flash,70:30 setting
2,AMR00103,AMRU5B-01ba,2045,Joshua Keener,04/10/2051,Rhodes MS/NMR,27/09/2051,13.6,Y,AMRU5B-01b,...,DCM: MeOH 2:1,InterestPC1,Mr,115.1,Y,Y,https://doi.org/comp4,DCM/MeOH,SPE fraction,None
3,AMR00104,MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,Rhodes Bioassay/Rhodes MS,24/03/2051,67.3,N,MOZ42-1JR-01a,...,DMSO,InterestPC8,five,26.9,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,SPE fraction,None
4,AMR00105,MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,Rhodes MS,12/07/2050,39.7,N,MOZ42-3RH-02a,...,"aq.MeOH, MeOH",InterestPC10,crime,215.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction from flash,None
5,AMR00106,AMRU5B-01ab,2045,Joshua Keener,04/10/2051,Membrane Disruption assay,27/09/2051,75.8,Y,AMRU5B-01a,...,DMSO,InterestPC9,reach,214.9,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,SPE fraction,None
6,AMR00107,1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,42.5,N,1-1*#2R2-01c,...,DCM: MeOH 2:1,InterestPC8,father,206.6,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,Fraction from flash,None
7,AMR00108,MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,Membrane Disruption assay,24/03/2051,9.6,N,MOZ42-1JR-01b,...,"aq.MeOH, MeOH",N/A,really,214.2,Y,Y,https://doi.org/comp4,DCM/MeOH,Fraction from flash,Used compound G in fractionation
8,AMR00109,SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,Membrane Disruption assay,17/11/2050,12.9,N,SAF43-5RH-01b,...,"aq.MeOH, MeOH",InterestPC4,religious,204.7,N,Y,Y,DCM:MeOH 2:1,SPE fraction,20:80 setting
9,AMR00110,SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,Rhodes Bioassay,25/08/2054,28.7,Y,SAF43-12JR-01a,...,DCM: MeOH 2:1,InterestPC2,agent,20.1,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,Used compound G in fractionation


In [400]:
# Add PC to the start of all supplierIDs
pure_df['supplierID'] = 'PC-' + pure_df['supplierID']
pure_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR00101,PC-MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,Rhodes Bioassay/Rhodes MS,25/03/2055,88.2,N,MOZ42-13RH-02b,...,DMSO,InterestPC5,former,126.2,N,Y,https://doi.org/comp3,MeOH,SPE fraction,Used compound G in fractionation
1,AMR00102,PC-1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,29.4,Y,1-1*#2R2-02a,...,"aq.MeOH, MeOH",InterestPC10,fish,280.6,Y,Y,Y,MeOH,Fraction from flash,70:30 setting
2,AMR00103,PC-AMRU5B-01ba,2045,Joshua Keener,04/10/2051,Rhodes MS/NMR,27/09/2051,13.6,Y,AMRU5B-01b,...,DCM: MeOH 2:1,InterestPC1,Mr,115.1,Y,Y,https://doi.org/comp4,DCM/MeOH,SPE fraction,None
3,AMR00104,PC-MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,Rhodes Bioassay/Rhodes MS,24/03/2051,67.3,N,MOZ42-1JR-01a,...,DMSO,InterestPC8,five,26.9,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,SPE fraction,None
4,AMR00105,PC-MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,Rhodes MS,12/07/2050,39.7,N,MOZ42-3RH-02a,...,"aq.MeOH, MeOH",InterestPC10,crime,215.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction from flash,None
5,AMR00106,PC-AMRU5B-01ab,2045,Joshua Keener,04/10/2051,Membrane Disruption assay,27/09/2051,75.8,Y,AMRU5B-01a,...,DMSO,InterestPC9,reach,214.9,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,SPE fraction,None
6,AMR00107,PC-1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,42.5,N,1-1*#2R2-01c,...,DCM: MeOH 2:1,InterestPC8,father,206.6,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,Fraction from flash,None
7,AMR00108,PC-MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,Membrane Disruption assay,24/03/2051,9.6,N,MOZ42-1JR-01b,...,"aq.MeOH, MeOH",N/A,really,214.2,Y,Y,https://doi.org/comp4,DCM/MeOH,Fraction from flash,Used compound G in fractionation
8,AMR00109,PC-SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,Membrane Disruption assay,17/11/2050,12.9,N,SAF43-5RH-01b,...,"aq.MeOH, MeOH",InterestPC4,religious,204.7,N,Y,Y,DCM:MeOH 2:1,SPE fraction,20:80 setting
9,AMR00110,PC-SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,Rhodes Bioassay,25/08/2054,28.7,Y,SAF43-12JR-01a,...,DCM: MeOH 2:1,InterestPC2,agent,20.1,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,Used compound G in fractionation


In [401]:
# Make InterestPC4 and N/A have no existing literature and no existing patent
pure_df.loc[pure_df['stereoComments'] == 'InterestPC4', 'existingPatent'] = 'N'
pure_df.loc[pure_df['stereoComments'] == 'InterestPC4', 'existingLiterature'] = 'N'
pure_df.loc[pure_df['stereoComments'] == 'N/A', 'existingPatent'] = 'N'
pure_df.loc[pure_df['stereoComments'] == 'N/A', 'existingLiterature'] = 'N'
pure_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR00101,PC-MOZ42-13RH-02ba,2042,Joshua Keener,01/04/2055,Rhodes Bioassay/Rhodes MS,25/03/2055,88.2,N,MOZ42-13RH-02b,...,DMSO,InterestPC5,former,126.2,N,Y,https://doi.org/comp3,MeOH,SPE fraction,Used compound G in fractionation
1,AMR00102,PC-1-1*#2R2-02aa,2046,Joshua Keener,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,29.4,Y,1-1*#2R2-02a,...,"aq.MeOH, MeOH",InterestPC10,fish,280.6,Y,Y,Y,MeOH,Fraction from flash,70:30 setting
2,AMR00103,PC-AMRU5B-01ba,2045,Joshua Keener,04/10/2051,Rhodes MS/NMR,27/09/2051,13.6,Y,AMRU5B-01b,...,DCM: MeOH 2:1,InterestPC1,Mr,115.1,Y,Y,https://doi.org/comp4,DCM/MeOH,SPE fraction,None
3,AMR00104,PC-MOZ42-1JR-01ab,2042,Ruby Johnson,31/03/2051,Rhodes Bioassay/Rhodes MS,24/03/2051,67.3,N,MOZ42-1JR-01a,...,DMSO,InterestPC8,five,26.9,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,SPE fraction,None
4,AMR00105,PC-MOZ42-3RH-02aa,2042,Joshua Keener,19/07/2050,Rhodes MS,12/07/2050,39.7,N,MOZ42-3RH-02a,...,"aq.MeOH, MeOH",InterestPC10,crime,215.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction from flash,None
5,AMR00106,PC-AMRU5B-01ab,2045,Joshua Keener,04/10/2051,Membrane Disruption assay,27/09/2051,75.8,Y,AMRU5B-01a,...,DMSO,InterestPC9,reach,214.9,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,SPE fraction,None
6,AMR00107,PC-1-1*#2R2-01ca,2046,Ruby Johnson,30/01/2052,Rhodes Bioassay/Rhodes MS,23/01/2052,42.5,N,1-1*#2R2-01c,...,DCM: MeOH 2:1,InterestPC8,father,206.6,N,Y,https://doi.org/comp3,DCM:MeOH 2:1,Fraction from flash,None
7,AMR00108,PC-MOZ42-1JR-01ba,2042,Joshua Keener,31/03/2051,Membrane Disruption assay,24/03/2051,9.6,N,MOZ42-1JR-01b,...,"aq.MeOH, MeOH",N/A,really,214.2,Y,N,N,DCM/MeOH,Fraction from flash,Used compound G in fractionation
8,AMR00109,PC-SAF43-5RH-01ba,2043,Ruby Johnson,24/11/2050,Membrane Disruption assay,17/11/2050,12.9,N,SAF43-5RH-01b,...,"aq.MeOH, MeOH",InterestPC4,religious,204.7,N,N,N,DCM:MeOH 2:1,SPE fraction,20:80 setting
9,AMR00110,PC-SAF43-12JR-01ab,2043,Ruby Johnson,01/09/2054,Rhodes Bioassay,25/08/2054,28.7,Y,SAF43-12JR-01a,...,DCM: MeOH 2:1,InterestPC2,agent,20.1,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,Used compound G in fractionation


### Make MNP and AMR dataframes

In [402]:
# MNP spreadsheet: fractionFirst, fractionSecond, records not pure from fractionThird, and records not pure from fractionFourth
mnp_df = fractionFirst_df.append(fractionSecond_df, ignore_index=True)
mnp_df = mnp_df.append(fractionThird_df, ignore_index=True)
mnp_df = mnp_df.append(fractionFourth_df, ignore_index=True)
mnp_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2236908492.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mnp_df = fractionFirst_df.append(fractionSecond_df, ignore_index=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2236908492.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mnp_df = mnp_df.append(fractionThird_df, ignore_index=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2236908492.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mnp_df = mnp_df.append(fractionFourth_df, ignore_index=True)


,mnpID,supplierID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00001,TIC2041-3RH-01,96.6,2041,Joshua Keener,10/02/2052,RU LC-MS & 1H-NMR,03/02/2052,Fraction from flash,1.00,DCM/MeOH,Marine,TIC2041-3RH,realize,RES2041,Roodt,Ruby Johnson,1,None
1,MNP00002,TIC2041-8RJ-01,44.1,2041,Ruby Johnson,14/12/2053,RU LC-MS & 1H-NMR,07/12/2053,SPE fraction,20.00,DCM/MeOH,Marine,TIC2041-8RJ,discuss,RES2041,Roodt,Priscilla Evans,1,None
2,MNP00003,TIC2041-2JR-01,0.7,2041,Ruby Johnson,18/01/2055,CD Experiment,11/01/2055,Fraction from flash,0.50,DCM/MeOH,Marine,TIC2041-2JR,though,RES2041,Smith,Priscilla Evans,1,None
3,MNP00004,AMRU3B-01,91.1,2045,Joshua Keener,15/01/2051,RU Bioassay,08/01/2051,SPE fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU3B,day,RES2045,Smith,Priscilla Evans,1,50:50 setting
4,MNP00005,SAF43-5RH-01,93.9,2043,Ruby Johnson,29/09/2050,Rhodes MS,22/09/2050,Fraction from flash,0.10,DCM/MeOH,Marine,SAF43-5RH,attorney,RES2043,Smith,Amanda Bowker,1,20:20 setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,MNP00256,1-1*#3R1-01aab,55.9,2046,Ruby Johnson,12/02/2055,RU Bioassay,05/02/2055,Fraction from flash,0.25,DCM:MeOH 2:1,Marine,1-1*#3R1-01aa,know,RES2046,Smith,Priscilla Evans,4,None
256,MNP00257,AMRU4A-01baa,72.2,2047,Ruby Johnson,23/08/2051,RU Bioassay,16/08/2051,Fraction,0.25,DCM:MeOH 2:1,Microbial,AMRU4A-01ba,myself,RES2047,Roodt,Priscilla Evans,4,None
257,MNP00258,AMRU1C-01aac,52.7,2045,Joshua Keener,22/01/2055,CD Experiment,15/01/2055,HPLC Fraction,0.10,DCM/MeOH,Microbial,AMRU1C-01aa,bar,RES2045,Roodt,Priscilla Evans,4,None
258,MNP00259,AMRU7B-02aab,70.0,2045,Joshua Keener,06/05/2051,RU Bioassay,29/04/2051,HPLC Fraction,50.00,DCM/MeOH,Microbial,AMRU7B-02aa,company,RES2045,Roodt,Ruby Johnson,4,Used compound Q in fractionation


In [403]:
original_df.drop(['submitterInitials'], axis=1, inplace=True)
amr_df = pd.concat([original_df, pure_df], ignore_index=True)
amr_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes MS,01/10/2053,16.3,N/A,AAP_1,...,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice4,Methanol,NaN,NaN
1,AMR00002,MOZ42-1JR,2042,Joshua Keener,06/01/2051,Rhodes MS,30/12/2050,49.9,N/A,MOZ42-1,...,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,NaN,NaN
2,AMR00003,TIC2041-1RH,2041,Ruby Johnson,31/01/2050,Rhodes MS/NMR,24/01/2050,94.0,N/A,TIC2041-1,...,N/A,N/A,N/A,N/A,N,N,Y,DMSO,NaN,NaN
3,AMR00004,MOZ42-2JR,2042,Ruby Johnson,16/10/2050,Rhodes MS/NMR,09/10/2050,95.0,N/A,MOZ42-2,...,N/A,N/A,N/A,N/A,N,N,Y,DMSO,NaN,NaN
4,AMR00005,TIC2041-2JR,2041,Joshua Keener,21/12/2054,"Rhodes Bioassay, PK assay",14/12/2054,14.0,N/A,TIC2041-2,...,N/A,N/A,N/A,N/A,N,N,Y,DMSO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,AMR00156,PC-AMRU3B-02bbd,2045,Ruby Johnson,09/04/2051,Rhodes LC/MS + NMR & Bioassay,02/04/2051,18.4,Y,AMRU3B-02bb,...,"aq.MeOH, MeOH",InterestPC6,citizen,324.3,N,Y,https://doi.org/comp4,DCM:MeOH 2:1,Fraction from flash,None
156,AMR00157,PC-AMRU7B-02aaa,2045,Joshua Keener,06/05/2051,Membrane Disruption assay,29/04/2051,11.4,N,AMRU7B-02aa,...,DCM: MeOH 2:1,InterestPC3,treatment,70.9,Y,Y,Y,DCM/MeOH,HPLC Fraction,Used compound Q in fractionation
157,AMR00158,PC-AMRU3B-02bba,2045,Ruby Johnson,09/04/2051,"Rhodes Bioassay, PK assay",02/04/2051,45.6,N,AMRU3B-02bb,...,DMSO,InterestPC6,size,26.9,Y,Y,https://doi.org/comp1,DCM:MeOH 2:1,Fraction from flash,None
158,AMR00159,PC-MOZ42-13RH-01aba,2042,Joshua Keener,29/04/2055,Rhodes MS,22/04/2055,83.1,N,MOZ42-13RH-01ab,...,DMSO,InterestPC7,live,64.1,Y,Y,https://doi.org/comp3,DCM:MeOH 2:1,SPE fraction,None


### Strain Spreadsheet

In [404]:
strain_df = amr_df.copy(deep=True)
new_strain = strain_df.loc[strain_df['sourceType'] == "Bacterial"]
new_strain = new_strain[['supplierID']].reset_index(drop=True)
new_strain

,supplierID
0,AAP_1
1,AAP_2
2,AAP_3
3,AAP_4
4,AAP_5
5,AAP_6
6,AAP_7
7,AAP_8
8,AAP_9
9,AAP_10


In [405]:
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    cultivations = ['MEM', 'WYM', 'PDM', 'PDM+I/O']

    def cultivationMedia(self):
        return random.choice(self.cultivations)

# Add the Provider to our faker object
fake.add_provider(Provider)

# Change:
# AMR ID, supplierID, destination, synthesis, sourceType, pure, ms, nmr, researchGroup, solubility, 
# stereoComments, SMILE, mw, additionalMetadata, existingPatent, existingLiterature

def create_strain_data(fake, no_of_rows):
    columns = ['cultivationMedia']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

strain_df = create_strain_data(fake, len(new_strain))
strain_df = pd.concat([new_strain, strain_df], axis=1)
strain_df

,supplierID,cultivationMedia
0,AAP_1,PDM
1,AAP_2,PDM
2,AAP_3,PDM
3,AAP_4,WYM
4,AAP_5,PDM
5,AAP_6,MEM
6,AAP_7,PDM+I/O
7,AAP_8,MEM
8,AAP_9,PDM
9,AAP_10,PDM


### Bioassay spreadsheet

processID -> bioassay

responsiblePi -> bioassay

researcherId -> bioassay + person

dateReceived -> bioassay

sampleID -> associated with extract/fraction (repeated)

sourceSpecimen -> associated with extract/fraction (repeated)

molecularID -> bioassay

cultivationMedia -> strain

diluent -> bioassay

concentration -> bioassay

volume -> removed

amount -> bioassay

all other values (ecoli_viability to hppk_activity) -> bioassay

some have been removed

In [406]:
# Get 150 records from both amr and mnp spreadsheets
# Need amr/mnpID, dateAssigned, species
amr_proc = amr_df[["amrID", "researchGroup", "submittedBy", "assignedDate", "species", "destinationOfSample"]]
amr_proc.rename(columns={'amrID':'processID'}, inplace=True)
mnp_proc = mnp_df[["mnpID", "researchGroup", "submittedBy", "assignedDate", "species", "destinationOfSample"]]
mnp_proc.rename(columns={'mnpID':'processID'}, inplace=True)
proc_df = pd.concat([amr_proc, mnp_proc], ignore_index=True)
# Get both bioassay and pk
df_all = proc_df[proc_df.destinationOfSample.str.contains("Bioassay", regex=False)].append(proc_df[proc_df.destinationOfSample.str.contains("PK", regex=False)])
df_all.drop_duplicates(keep='first', inplace=True)
# Get only pk
pks = proc_df[proc_df.destinationOfSample.str.contains("PK", regex=False)]
# Find values where indexes are same, and put True in pk_requested
bioassay_df = df_all.merge(pks.drop_duplicates(), on=["processID", "researchGroup", "submittedBy", "assignedDate", "species", "destinationOfSample"], how='left', indicator=True)
bioassay_df['_merge'] = (bioassay_df['_merge'] == 'both')
bioassay_df.rename(columns={'_merge':'pk_requested'}, inplace=True)

# bioFirst_df = amr_df.sample(n=60, random_state=1234, ignore_index=True)
# bioFirst_df = bioFirst_df[["amrID", "researchGroup", "submittedBy", "assignedDate", "species"]]
# bioFirst_df.rename(columns={'amrID':'processID'}, inplace=True)
# bioSecond_df = mnp_df.sample(n=40, random_state=1234, ignore_index=True)
# bioSecond_df = bioSecond_df[["mnpID", "researchGroup", "submittedBy", "assignedDate", "species"]]
# bioSecond_df.rename(columns={'mnpID':'processID'}, inplace=True)
# bioassay_df = pd.concat([bioFirst_df, bioSecond_df], ignore_index=True)
bioassay_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2027117025.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amr_proc.rename(columns={'amrID':'processID'}, inplace=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2027117025.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mnp_proc.rename(columns={'mnpID':'processID'}, inplace=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\2027117025.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = proc_df[proc_df.destinationOfSample.str.contains("Bioassay", regex=

,processID,researchGroup,submittedBy,assignedDate,species,destinationOfSample,pk_requested
0,AMR00005,Smith,Julia Reed,21/12/2054,though,"Rhodes Bioassay, PK assay",True
1,AMR00006,Preston,Robert Hobson,26/04/2050,memory,Rhodes LC/MS + NMR & Bioassay,False
2,AMR00008,Preston,Julia Reed,09/11/2050,television,"Rhodes Bioassay, PK assay",True
3,AMR00010,Wellington,Robert Hobson,13/01/2052,realize,Rhodes Bioassay,False
4,AMR00011,Smith,Ruby Johnson,03/03/2050,especially,Rhodes LC/MS + NMR & Bioassay,False
...,...,...,...,...,...,...,...
131,MNP00233,Smith,Amanda Bowker,05/12/2050,structure,RU Bioassay,False
132,MNP00234,Roodt,Ruby Johnson,23/08/2051,myself,RU Bioassay,False
133,MNP00256,Smith,Priscilla Evans,12/02/2055,know,RU Bioassay,False
134,MNP00257,Roodt,Priscilla Evans,23/08/2051,myself,RU Bioassay,False


In [407]:
class Provider(BaseProvider):
    solvents = ['MeOH', 'Dried down']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]

    def diluent(self):
        return random.choice(self.solvents)
    
    def concentration(self):
        return random.choice(self.concentrations)

    def amount(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))

    def ecoliV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def ecoliSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def saureusV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def saureusSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def pareuV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def pareuSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def saureusBioV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def saureusBioSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def pareuBioV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def pareuBioSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def cytotoxV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def cytotoxSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def pkActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def dxrActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def confirm_dxrActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def hppkActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    

# Add the Provider to our faker object
fake.add_provider(Provider)

# Need: responsiblePI, researcher, dateReceived, diluent, concentration, amount, ecoliV, ecoliSD, saureusV, saureusSD, pareuV, pareuSD, saureusBioV, saureusBioSD,
# paureuBioV, paureuBioSD, cytotoxV, cytotoxSD, pkActivity, dxrActivity, confirm_dxrActivity, hppkActivity


def create_bioassay_data(fake, no_of_rows):
    columns = ['diluent', 'concentration', 'amount', 'ecoliV', 'ecoliSD', 'saureusV', 'saureusSD', 'pareuV', 'pareuSD', 'saureusBioV', 'saureusBioSD',
                'pareuBioV', 'pareuBioSD', 'cytotoxV', 'cytotoxSD', 'pkActivity', 'dxrActivity', 'confirm_dxrActivity', 'hppkActivity']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

rest_bio_df = create_bioassay_data(fake, len(bioassay_df))
rest_bio_df

,diluent,concentration,amount,ecoliV,ecoliSD,saureusV,saureusSD,pareuV,pareuSD,saureusBioV,saureusBioSD,pareuBioV,pareuBioSD,cytotoxV,cytotoxSD,pkActivity,dxrActivity,confirm_dxrActivity,hppkActivity
0,MeOH,0.50,9.9,70.4,7.1,7.9,0.2,92.0,3.6,95.2,7.2,16.9,8.1,24.9,1.0,100.6,82.6,4.4,108.1
1,Dried down,50.00,9.7,69.1,3.4,18.4,6.1,82.6,8.7,32.0,3.0,15.7,3.8,90.3,6.6,37.4,100.3,9.9,53.7
2,Dried down,0.10,1.3,82.1,1.0,77.2,1.0,34.8,5.0,39.3,3.3,30.6,5.0,6.3,7.1,7.1,71.5,43.8,101.9
3,MeOH,1.00,5.2,46.8,7.3,107.9,7.2,53.7,2.5,22.9,1.3,97.9,8.0,14.7,7.2,77.4,52.8,35.6,78.7
4,Dried down,0.50,2.3,70.6,9.9,65.9,5.1,40.4,0.9,36.3,1.9,25.9,5.2,33.2,3.2,99.3,44.5,22.0,37.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,MeOH,0.10,2.3,101.0,0.6,32.5,7.5,43.1,0.7,15.3,3.5,74.4,9.0,89.5,7.2,99.2,69.7,23.0,59.6
132,MeOH,1.00,4.1,31.9,6.5,84.5,2.4,99.4,8.0,86.2,7.0,87.4,3.7,51.6,0.3,13.1,69.2,27.8,98.3
133,Dried down,50.00,5.0,92.4,1.0,103.4,9.7,2.1,6.4,28.2,7.7,73.2,2.4,85.4,2.5,67.4,31.7,0.1,31.5
134,MeOH,1.00,2.5,85.7,7.6,11.7,3.2,0.2,4.8,72.9,2.4,70.9,0.8,95.4,4.0,23.8,78.4,106.4,28.5


In [408]:
bio_df = pd.concat([bioassay_df, rest_bio_df], axis=1)
bio_df.rename(columns={'submittedBy':'researcher', 'species':'molecularID', 'assignedDate':'dateReceived', 'researchGroup':'responsiblePI'}, inplace=True)
bio_df

,processID,responsiblePI,researcher,dateReceived,molecularID,destinationOfSample,pk_requested,diluent,concentration,amount,...,saureusBioV,saureusBioSD,pareuBioV,pareuBioSD,cytotoxV,cytotoxSD,pkActivity,dxrActivity,confirm_dxrActivity,hppkActivity
0,AMR00005,Smith,Julia Reed,21/12/2054,though,"Rhodes Bioassay, PK assay",True,MeOH,0.50,9.9,...,95.2,7.2,16.9,8.1,24.9,1.0,100.6,82.6,4.4,108.1
1,AMR00006,Preston,Robert Hobson,26/04/2050,memory,Rhodes LC/MS + NMR & Bioassay,False,Dried down,50.00,9.7,...,32.0,3.0,15.7,3.8,90.3,6.6,37.4,100.3,9.9,53.7
2,AMR00008,Preston,Julia Reed,09/11/2050,television,"Rhodes Bioassay, PK assay",True,Dried down,0.10,1.3,...,39.3,3.3,30.6,5.0,6.3,7.1,7.1,71.5,43.8,101.9
3,AMR00010,Wellington,Robert Hobson,13/01/2052,realize,Rhodes Bioassay,False,MeOH,1.00,5.2,...,22.9,1.3,97.9,8.0,14.7,7.2,77.4,52.8,35.6,78.7
4,AMR00011,Smith,Ruby Johnson,03/03/2050,especially,Rhodes LC/MS + NMR & Bioassay,False,Dried down,0.50,2.3,...,36.3,1.9,25.9,5.2,33.2,3.2,99.3,44.5,22.0,37.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,MNP00233,Smith,Amanda Bowker,05/12/2050,structure,RU Bioassay,False,MeOH,0.10,2.3,...,15.3,3.5,74.4,9.0,89.5,7.2,99.2,69.7,23.0,59.6
132,MNP00234,Roodt,Ruby Johnson,23/08/2051,myself,RU Bioassay,False,MeOH,1.00,4.1,...,86.2,7.0,87.4,3.7,51.6,0.3,13.1,69.2,27.8,98.3
133,MNP00256,Smith,Priscilla Evans,12/02/2055,know,RU Bioassay,False,Dried down,50.00,5.0,...,28.2,7.7,73.2,2.4,85.4,2.5,67.4,31.7,0.1,31.5
134,MNP00257,Roodt,Priscilla Evans,23/08/2051,myself,RU Bioassay,False,MeOH,1.00,2.5,...,72.9,2.4,70.9,0.8,95.4,4.0,23.8,78.4,106.4,28.5


In [409]:
# Turn off pk activity for those not requested
bio_df.loc[bio_df['pk_requested'] == False, 'pkActivity'] = None
# Remove destinationOfSample, pk_requested
bio_df.drop(['destinationOfSample', 'pk_requested'], axis=1, inplace=True)
bio_df

,processID,responsiblePI,researcher,dateReceived,molecularID,diluent,concentration,amount,ecoliV,ecoliSD,...,saureusBioV,saureusBioSD,pareuBioV,pareuBioSD,cytotoxV,cytotoxSD,pkActivity,dxrActivity,confirm_dxrActivity,hppkActivity
0,AMR00005,Smith,Julia Reed,21/12/2054,though,MeOH,0.50,9.9,70.4,7.1,...,95.2,7.2,16.9,8.1,24.9,1.0,100.6,82.6,4.4,108.1
1,AMR00006,Preston,Robert Hobson,26/04/2050,memory,Dried down,50.00,9.7,69.1,3.4,...,32.0,3.0,15.7,3.8,90.3,6.6,NaN,100.3,9.9,53.7
2,AMR00008,Preston,Julia Reed,09/11/2050,television,Dried down,0.10,1.3,82.1,1.0,...,39.3,3.3,30.6,5.0,6.3,7.1,7.1,71.5,43.8,101.9
3,AMR00010,Wellington,Robert Hobson,13/01/2052,realize,MeOH,1.00,5.2,46.8,7.3,...,22.9,1.3,97.9,8.0,14.7,7.2,NaN,52.8,35.6,78.7
4,AMR00011,Smith,Ruby Johnson,03/03/2050,especially,Dried down,0.50,2.3,70.6,9.9,...,36.3,1.9,25.9,5.2,33.2,3.2,NaN,44.5,22.0,37.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,MNP00233,Smith,Amanda Bowker,05/12/2050,structure,MeOH,0.10,2.3,101.0,0.6,...,15.3,3.5,74.4,9.0,89.5,7.2,NaN,69.7,23.0,59.6
132,MNP00234,Roodt,Ruby Johnson,23/08/2051,myself,MeOH,1.00,4.1,31.9,6.5,...,86.2,7.0,87.4,3.7,51.6,0.3,NaN,69.2,27.8,98.3
133,MNP00256,Smith,Priscilla Evans,12/02/2055,know,Dried down,50.00,5.0,92.4,1.0,...,28.2,7.7,73.2,2.4,85.4,2.5,NaN,31.7,0.1,31.5
134,MNP00257,Roodt,Priscilla Evans,23/08/2051,myself,MeOH,1.00,2.5,85.7,7.6,...,72.9,2.4,70.9,0.8,95.4,4.0,NaN,78.4,106.4,28.5


#### Collection spreadsheets

Sample ID -> collection

Common Name -> collection - comes with taxonomic data

Other IDs -> collection

Country -> country

Province -> province

City -> city

Local Area -> localarea

Station -> station

Date -> collection

Latitude -> site

Longitude -> site

Depth min -> site

Depth max -> site

Collected by -> person_collected

Identified by -> collection/person

Classification -> collection

Other description -> collection

Kingdom to Species - use actual data

In [410]:
# Years with collection information - 2041, 2042, 2043
collectTIC = original_df.loc[original_df['year'] == 2041]
collectMOZ = original_df.loc[original_df['year'] == 2042]
collectSAF = original_df.loc[original_df['year'] == 2043]
collection_df = pd.concat([collectTIC, collectMOZ, collectSAF], ignore_index=True)
collection_df = collection_df[["sourceID", "permitNo"]]
collection_df

,sourceID,permitNo
0,TIC2041-1,RES2041
1,TIC2041-2,RES2041
2,TIC2041-3,RES2041
3,TIC2041-4,RES2041
4,TIC2041-5,RES2041
5,TIC2041-6,RES2041
6,TIC2041-7,RES2041
7,TIC2041-8,RES2041
8,TIC2041-9,RES2041
9,TIC2041-10,RES2041


In [411]:
# Faker for Country, collected by, identified by, classification
fake = Faker()
Faker.seed(1235)
random.seed(1235)

class Provider(BaseProvider):
    collectors = ['Mandy Peterson', 'Clive Morrison', 'Trevor Andrews']
    collectorsTwo = ['Mandy Peterson', 'Clive Morrison', 'Trevor Andrews']
    identifiers = ['Clive Morrison', 'Harry Warner']
    classifications = ['Preliminary' ,'Complete', 'Questionable', 'Tentative']
    
    def collectedByOne(self):
        return random.choice(self.collectors)

    def collectedByTwo(self):
        return random.choice(self.collectorsTwo)

    def identifiedBy(self):
        return random.choice(self.identifiers)
    
    def classification(self):
        return random.choice(self.classifications)
    
# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_collection_data(fake, no_of_rows):
    columns = ['collectedByOne', 'collectedByTwo', 'identifiedBy', 'classification']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

fakerCollection_df = create_collection_data(fake, len(collection_df))
# Make collectedBy and collectedByTwo one column

collectedOneList = fakerCollection_df['collectedByOne'].tolist()
collectedTwoList = fakerCollection_df['collectedByTwo'].tolist()
collectedTogether = []

for one, two in zip(collectedOneList, collectedTwoList):
    if one == two:
        newStr = one
    elif one == "Clive Morrison" or two == "Clive Morrison":
        newStr = one + " and " + two
    else:
        newStr = one + "; " + two
    collectedTogether.append(newStr)

togetherSeries = pd.Series(collectedTogether)
fakerCollection_df.insert(1, 'collectedBy', togetherSeries)
fakerCollection_df.drop(["collectedByOne", "collectedByTwo"], axis=1, inplace=True)
fakerCollection_df

,collectedBy,identifiedBy,classification
0,Trevor Andrews and Clive Morrison,Clive Morrison,Preliminary
1,Clive Morrison,Clive Morrison,Tentative
2,Trevor Andrews; Mandy Peterson,Harry Warner,Preliminary
3,Trevor Andrews and Clive Morrison,Harry Warner,Complete
4,Clive Morrison and Mandy Peterson,Clive Morrison,Questionable
5,Clive Morrison,Harry Warner,Questionable
6,Mandy Peterson and Clive Morrison,Harry Warner,Preliminary
7,Trevor Andrews,Clive Morrison,Tentative
8,Clive Morrison and Trevor Andrews,Clive Morrison,Questionable
9,Trevor Andrews and Clive Morrison,Harry Warner,Tentative


In [412]:
# Get the full location data for each country
# South Africa - TIC, Mozambique - MOZ, Seychelles (for SAF, even though not in real life)
# Make lists for strings to use in locations

def makeCountryLocation(numLocals, givenSeed):
    random.seed(givenSeed)
    local = []
    latitude = []
    longitude = []
    for i in range(numLocals):
        local.append(fake.word().capitalize() + " " + fake.word())
        latitude.append(round(random.uniform( -90,  90), 4))
        longitude.append(round(random.uniform(-180, 180), 5))

    station = []
    for i in range(numLocals * 2):
        station.append(fake.word().capitalize() + " " + fake.word())
        if i % 2 == 0:
            lat = latitude[i//2] - round(random.uniform(0, 1), 4)
            long = longitude[i//2] - round(random.uniform(0, 1), 4)
            latitude.append(round(lat, 4))
            longitude.append(round(long, 4))

    min = []
    max = []
    for i in range(numLocals * 4):
        min.append(random.randint(0, 15))
        max.append(min[i] + random.randint(4, 15))
        if i % 2 == 0:
            lat = latitude[i//2] + round(random.uniform(0, 0.1), 4)
            long = longitude[i//2] + round(random.uniform(0, 0.1), 4)
            latitude.append(round(lat, 4))
            longitude.append(round(long, 4))

    return local, station, latitude, longitude, min, max

# Uses strings to make up full locations
def makeLocations(numRecords, localList, stationList, latitudeList, longitudeList, minList, maxList):
    local = []
    station = []
    latitude = []
    longitude = []
    min = []
    max = []
    for i in range(numRecords):
        local.append(localList[i % len(localList)])
        station.append(stationList[i % len(stationList)])
        latitude.append(latitudeList[i % len(latitudeList)])
        longitude.append(longitudeList[i % len(longitudeList)])
        min.append(minList[i % len(minList)])
        max.append(maxList[i % len(maxList)])

    return (pd.DataFrame({'localArea':local, 'station':station, 'latitude':latitude, 'longitude':longitude, 'depthMin':min, 'depthMax':max}))
        
local, station, latitude, longitude, min, max = makeCountryLocation(2, 1234)
SAlocation_df = makeLocations(len(collectTIC), local, station, latitude, longitude, min, max)
SAlocation_df.insert(0, 'country', 'South Africa')
SAlocation_df.insert(1, 'region', 'Eastern Cape')
SAlocation_df.insert(2, 'city', fake.word().capitalize())
local, station, latitude, longitude, min, max = makeCountryLocation(2, 1233)
MOZlocation_df = makeLocations(len(collectMOZ), local, station, latitude, longitude, min, max)
MOZlocation_df.insert(0, 'country', 'Mozambique')
MOZlocation_df.insert(1, 'region', 'Gaza')
MOZlocation_df.insert(2, 'city', fake.word().capitalize())
local, station, latitude, longitude, min, max = makeCountryLocation(2, 1232)
SAFlocation_df = makeLocations(len(collectSAF), local, station, latitude, longitude, min, max)
SAFlocation_df.insert(0, 'country', 'Seychelles')
SAFlocation_df.insert(1, 'region', 'Bel Air')
SAFlocation_df.insert(2, 'city', fake.word().capitalize())
location_df = pd.concat([SAlocation_df, MOZlocation_df, SAFlocation_df], ignore_index=True)
location_df

,country,region,city,localArea,station,latitude,longitude,depthMin,depthMax
0,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9616,-21.33626,11,18
1,South Africa,Eastern Cape,Race,Language north,Many include,-88.6515,147.95135,0,9
2,South Africa,Eastern Cape,Race,When risk,Church second,83.0223,-21.91850,15,28
3,South Africa,Eastern Cape,Race,Language north,Happy set,-89.3231,147.86740,3,7
4,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9633,-21.23760,15,22
5,South Africa,Eastern Cape,Race,Language north,Many include,-88.6051,147.96040,14,19
6,South Africa,Eastern Cape,Race,When risk,Church second,83.0288,-21.86440,2,14
7,South Africa,Eastern Cape,Race,Language north,Happy set,-89.2652,147.89440,2,16
8,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9616,-21.33626,11,18
9,South Africa,Eastern Cape,Race,Language north,Many include,-88.6515,147.95135,0,9


In [413]:
tax_df = pd.read_excel("taxonomy.xlsx")
tax_df = tax_df.head(len(location_df))

In [414]:
Faker.seed(1234)
# Combining what we have together:
collection = pd.concat([collection_df, location_df, fakerCollection_df, tax_df], axis=1)
collection.insert(1, 'otherIDs', 'NaN')
collection.insert(15, 'otherDescription', 'NaN')
date = []
for i in range(len(collectTIC)):
    date.append(fake.date_between_dates(date_start = datetime(2041, 1, 1), date_end = datetime(2041, 12, 31)).strftime('%d/%m/%Y'))
for i in range(len(collectMOZ)):
    date.append(fake.date_between_dates(date_start = datetime(2042, 1, 1), date_end = datetime(2042, 12, 31)).strftime('%d/%m/%Y'))
for i in range(len(collectSAF)):
    date.append(fake.date_between_dates(date_start = datetime(2043, 1, 1), date_end = datetime(2043, 12, 31)).strftime('%d/%m/%Y'))
collection.insert(2, 'collectedDate', date)
collection

,sourceID,otherIDs,collectedDate,permitNo,country,region,city,localArea,station,latitude,...,Class,Subclass,Superorder,Order,Suborder,Family,Subfamily,Genus,Subgenus,Species
0,TIC2041-1,NaN,30/10/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9616,...,Ascidiacea,NaN,Enterogona,Aplousobranchia,NaN,Pseudodistomidae,NaN,Pseudodistoma,NaN,sp 015 RSA SPN roseum
1,TIC2041-2,NaN,21/06/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Many include,-88.6515,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Cyclacanthia,NaN,bellae
2,TIC2041-3,NaN,15/02/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Church second,83.0223,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 004 RSA nguni
3,TIC2041-4,NaN,03/01/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Happy set,-89.3231,...,Demospongiae,NaN,NaN,Dictyoceratida,NaN,Ircinidae,NaN,Psammocinia,NaN,sp 001 RSA MH NMMU
4,TIC2041-5,NaN,05/02/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9633,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 004 RSA nguni
5,TIC2041-6,NaN,20/12/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Many include,-88.6051,...,Demospongiae,NaN,NaN,Poecilosclerida,Myxillina,Hymedesmiidae,NaN,Phorbas,NaN,sp 001 RSA MKB CRRF
6,TIC2041-7,NaN,10/11/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Church second,83.0288,...,Demospongiae,NaN,NaN,Hadromerida,NaN,Clionaidae,NaN,NG Clionaidae 001 RSA MKB CRRF,NaN,sp 002 RSA SPN CRRF
7,TIC2041-8,NaN,15/08/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Happy set,-89.2652,...,Demospongiae,NaN,NaN,Hadromerida,NaN,Hemiasterellidae,NaN,Hemiasterella,NaN,sp001 thick purple fan
8,TIC2041-9,NaN,14/01/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9616,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Cyclacanthia,NaN,bellae
9,TIC2041-10,NaN,18/09/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Many include,-88.6515,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 001 RSA toufieki


### Converting to Excel spreadsheet

In [415]:
datatoexcelAMR = pd.ExcelWriter('TrialDummy1.xlsx')
datatoexcelMNP = pd.ExcelWriter('TrialDummy2.xlsx')
datatoexcelBioassay = pd.ExcelWriter('bioassayDummy.xlsx')
datatoexcelLocation = pd.ExcelWriter('location.xlsx')
datatoexcelCollection = pd.ExcelWriter('collection.xlsx')
datatoexcelStrain = pd.ExcelWriter('strain.xlsx')

amr_df.to_excel(datatoexcelAMR, index=False)
mnp_df.to_excel(datatoexcelMNP, index=False)
bio_df.to_excel(datatoexcelBioassay, index=False)
location_df.to_excel(datatoexcelLocation, index=False)
collection.to_excel(datatoexcelCollection, index=False)
strain_df.to_excel(datatoexcelStrain, index=False)

datatoexcelAMR.save()
datatoexcelMNP.save()
datatoexcelBioassay.save()
datatoexcelLocation.save()
datatoexcelCollection.save()
datatoexcelStrain.save()

C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\12166985.py:15: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelAMR.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\12166985.py:16: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelMNP.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\12166985.py:17: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelBioassay.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\12166985.py:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelLocation.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_21652\12166985.py:19: FutureWarning: save is not part of the public API, usage can giv